## Imports

In [ ]:
import os
import sys
import time
import importlib
import datetime
import json
import glob
import gzip
from pathlib import Path
from IPython.display import display
import numpy as np
from numpy.lib.recfunctions import join_by
from numpy.core.records import fromarrays
import matplotlib.pyplot as plt
from matplotlib.pyplot import pcolormesh
import pandas as pd
import aiohttp
import asyncpg
import math
import itertools
import edcompanion.timetools
import edcompanion.edsm_api
from edcompanion.timetools import make_datetime, make_naive_utc
import follow_log
def prettyprint(item):
    print(json.dumps(item, indent=4, sort_keys=False))
    
def record_to_dict(record):
    if isinstance(record,asyncpg.Record):
        return {k:v for k,v in record.items()}
    return {}

plt.rcParams["figure.figsize"] = (16,12)

pd.options.display.max_colwidth = 150

In [ ]:
pgsql_params = dict(
    dsn=os.getenv("PGSQL_URL"),
    server_settings={'search_path': "eddb"}
)
pgpool = await asyncpg.create_pool(**pgsql_params)

In [ ]:
pgpool = await asyncpg.create_pool(**pgsql_params)
print(await pgpool.fetch("SELECT * FROM systems WHERE name = $1", "Ix"))

In [ ]:
# https://www.edsm.net/api-logs-v1/get-position
importlib.reload(edcompanion.edsm_api)
from edcompanion.edsm_api import get_commander_position

system_name = get_commander_position('immerlicht', os.getenv('EDSM_TOKEN')).get('system')
print(system_name)

In [ ]:
importlib.reload(edcompanion.edsm_api)
from edcompanion.edsm_api import get_edsm_info

async def find_system(system, distance=40):
    if isinstance(system, str):
        q1 =  await pgpool.fetchrow(
            """
                SELECT s.*, 0 as distance, p.security
                FROM systems s
                LEFT JOIN populated p
                ON s.name = p.systemname
                where s.name = $1
            """, system)
        if not q1:
            return get_edsm_info(system)
        return q1
    
    assert len(system) == 3
    coordinates = system
    c20_location = [int(20*math.floor(v/20)) for v in coordinates]
    side = int(20*math.floor(distance/20))
    q1 = await pgpool.fetch(
        "SELECT systems.*, |/((x-$7)^2 + (y-$8)^2 + (z-$9)^2) as distance, populated.security "+
        "FROM systems "+
        "LEFT JOIN populated " +
        "ON systems.name = populated.systemname "
        "WHERE x>=$1 AND x<$2 AND  y>=$3 AND y<$4  AND  z>=$5 AND z<$6  AND |/((x-$7)^2 + (y-$8)^2 + (z-$9)^2) < $10"+
        "ORDER BY distance",
        *[d for c in coordinates for d in [c-40, c+40]], *coordinates, distance)
    if not q1:
        return q1
    return await find_system(q1[0].get("name"))

async def find_nearby_systems(system, distance, limit=5, include_neutron=True):
    if isinstance(system, str):
        ql = await find_system(system)
        coordinates = [ql.get(k) for k in ["x", "y","z"]]
    else:
        coordinates = system
        
    #c20_location = [int(20*math.floor(v/20)) for v in coordinates]
    side = int(20*math.ceil(distance/20))

    return await pgpool.fetch(
        "SELECT name, x,y,z, |/((x-$7)^2 + (y-$8)^2 + (z-$9)^2) as distance "+
        "FROM systems "+f"""
            WHERE  x>=$1 AND x<$2 
              AND  y>=$3 AND y<$4  
              AND  z>=$5 AND z<$6 
              {'AND NOT n' if not include_neutron else ''}
        """ +
        "  AND |/((x-$7)^2 + (y-$8)^2 + (z-$9)^2) < $10"+
        "ORDER BY distance LIMIT " + str(limit),
        *[d for c in coordinates for d in [c-side, c+side]], 
        *coordinates, distance)


edsm_info_cache={}

# async def get_edsm_info(edsm_session, sname):
#     if sname not in edsm_info_cache:
#         async with session.get('https://www.edsm.net/api-system-v1/bodies', params=dict(systemName=sname)) as req:
#             edsm_info_cache[sname] = await req.json()
#             if not edsm_info_cache.get(sname):
#                 edsm_info_cache[sname] = {}
#     return edsm_info_cache.get(sname,{})
    
    

In [ ]:
def auto_map(set_to_map, missing=set()):
    templist = sorted(set_to_map)
    return {s:i for i,s in zip(range(len(templist)), templist)}

def recreate_map(mapping):
    subval = min(mapping.values())
    if subval > 0:
        mapping = {k:v-subval for k,v in mapping.items()}

    return mapping

def create_automapper(start_dict):
    return lambda k: start_dict.get(
        k, 
        None if start_dict.update(
            dict(k=1+max(start_dict.values()))
        ) else start_dict.get(k))
        

## Constants

In [ ]:
raw_materials = {
    1:set(['carbon', 'vanadium', 'niobium', 'yttrium']),
    2:set(['chromium', 'phosphorus', 'molybdenum', 'technetium']),
    3:set(['sulphur', 'manganese' 'cadmium', 'ruthenium' ]),
    4:set(['zinc', 'selenium', 'iron', 'tin']),
    5:set(['germanium', 'nickel', 'tungsten', 'tellurium']),
    6:set(['polonium', 'arsenic', 'mercury', 'rhenium']),
    7:set(['antimony', 'zirconium', 'boron', 'lead'])
}


In [ ]:
body_types = sorted([T for T in {'Barycentre', 'Planet', 'Star'}])
body_types = {i:s for i,s in zip(range(len(body_types)), body_types)}

In [ ]:
body_types

In [ ]:
await find_system('Dalam')

In [ ]:
raw_materials

# Import logfiles

In [ ]:
{
    True : {}
}

In [ ]:
logpath = "/Users/fenke/Saved Games/Frontier Developments/Elite Dangerous"
logfiles = sorted(
    [os.path.join(logpath, f) for f in os.listdir(logpath) if 'Journal' in f.split('.')[0] and '.log' in f],
    key=lambda f:f.replace('-', '').replace('Journal.20', 'Journal.').replace('T','')
)

#logfiles = glob.glob(os.path.join(logpath, "journal.22*"))[-51:]

last_system = {
    "timestamp": "2000-01-01",
    "StarSystem": 'No System Found',
    "StarPos":[]
}
system_name = last_system["StarSystem"]
last_timestamp = make_naive_utc(make_datetime("2022-01-01")).timestamp()
session_timestamp = last_timestamp

systems = {}
system = {}
sold_systems = {}
bodies = {}
system_count = 0
jumps = {}
data = {}
events = {}
jump_info = {}
sessions = []
jumps = {}
factions = {}

journal_maps_spectral={'-':0}
journal_maps_luminosity={'-':0}
journal_maps_magnitude={'-':0}

map_spectral = create_automapper(journal_maps_spectral)
map_luminosity = create_automapper(journal_maps_luminosity)
map_magnitude = create_automapper(journal_maps_magnitude)


jump_events = set([
    "SupercruiseEntry",
    "FSDTarget"
    "StartJump",
    "FSDJump",
    "FuelScoop"
])

scan_events = set([
    "SAAScanComplete",
    "SAASignalsFound",
    "Scan",
    "FSSDiscoveryScan",
    "FSSSignalDiscovered",
    "FSSAllBodiesFound"
])

scan_types = set([
    "AutoScan",

])

excluded_events = set([
    #"Supercruise",
    "ReceiveText",
    "Location",
    "Commander",
    #"Materials",
    "Rank",
    "Progress",
    #"Reputation",
    "LoadGame",
    "EngineerProgress",
    "Music",
    #"Missions",
    "Loadout",
    "Music",
    "Statistics",
    "Cargo",
    #"SupercruiseEntry"
])
planet_values = {
    False: { # was-not-discovered
        False: {# was-not-mapped
            False: { # is-not-terraformable
                'Water world': 1559138,
                'Earthlike body': 4224870,
                'Ammonia world': 2242455,
            },
            True : { # is-terraformable
                'Water world': 4198704,
                "High metal content body": 2562654,
                "Rocky body": 2024270
            }
        },
        
        True: {# was-mapped, this specic combo is rubbish
        }
    },
    True: { # was-discovered
        False: {# was-not-mapped
            False: { # is-not-terraformable
                'Water world': 1312209,
                'Earthlike body': 3555753,
                'Ammonia world': 1887305,
            },
            True : { # is-terraformable
                'Water world': 3533732,
                "High metal content body": 2156792,
                "Rocky body": 1703675
            }
        },
        
        True: {# was-mapped
            False: { # is-not-terraformable
                'Water world': 540297,
                'Earthlike body': 1464068,
                'Ammonia world': 777091,
            },
            True : { # is-terraformable
                'Water world': 1455001,
                "High metal content body": 888051,
                "Rocky body": 701482
            }
        }
    }
}


async with pgpool.acquire() as pgconnection: 
    push_query = await pgconnection.prepare(
        """INSERT INTO eddb.systems (name, x, y, z) 
            VALUES ($1, $2, $3, $4) 
            ON CONFLICT DO NOTHING
        """
    )
    get_query = await pgconnection.prepare(
        """SELECT name, x, y, z
            FROM eddb.systems  
            WHERE name = $1
        """
    )

    async with aiohttp.ClientSession() as session:
    
        start = time.process_time()

        for filename in logfiles[-11:]:

            filesize=Path(filename).stat().st_size
            chunksize = 1 * 1024 * 1024
            est_count = int(filesize/chunksize) + 1
            sys.stdout.write(f"Reading {filename}, {round(filesize/(1024*1024),1)} Mb in approx {est_count} chunks\r")
            count = 0

            with open(filename, encoding="utf-8") as jsonfile:
                firstline = jsonfile.readline()

                while True:
                    count += 1
                    chunk = jsonfile.readlines(chunksize)
                    if chunk:
                        #data = []
                        for line in chunk:
                            if len(line) < 5:
                                continue
                            item = json.loads(line[0:-2]) if line[-2] == "," else json.loads(line)
                            if item.get("event") in excluded_events:
                                continue

                            timestamp = item.pop("timestamp")
                            eventname = item.get("event")

                            current_timestamp = make_datetime(timestamp).timestamp()
                            if round(current_timestamp - last_timestamp) > 900:
                                if last_timestamp-session_timestamp > 0:
                                    sessions.append([
                                        session_timestamp, 
                                        last_timestamp, 
                                        last_timestamp-session_timestamp])
                                    session_timestamp = current_timestamp

                            last_timestamp = current_timestamp
                            
                            if eventname in ["StartJump", "FSDJump", "Location", "SupercruiseEntry", "FSSDiscoveryScan", "Scan"]:
                                
                                system_name = item.get("StarSystem", system_name)
                                
                                if system_name not in jump_info:
                                    db_entry = await get_query.fetchrow(system_name)
                                    jump_info[system_name] = dict(
                                        bodyid=None,
                                        bodycount=None,
                                        bodies=dict(),
                                        #edsm=get_edsm_info(session, system_name) if db_entry else {},
                                    )
                                
                                if system_name not in systems:
                                    systems[system_name] = dict(bodies=dict(), factions=dict(), signals=set(), coord=[0,0,0], visits=0)

                                system = systems.get(system_name)
                                
                                for faction in item.get("Factions",[]) + [dict(Name=item.get('Faction','No Faction'))]:
                                    factionname = faction.get("Name")
                                    system.get("factions").update({factionname:faction.get('Influence',0)})
                                    f = factions.get(
                                        factionname,
                                        dict(systems=dict(), bounty_voucher='1970-01-01', bounty='1970-01-01', reputation=0))
                                    f.get('systems').update({item.get("StarSystem"):faction.get('Influence',0)})
                                    f.update({
                                        'allegiance':faction.get('Allegiance', ''),
                                        'reputation':faction.get('MyReputation', 0), 
                                        'state':faction.get('FactionState', f.get('state', ''))})
                                    factions[factionname] = f
                                    
                                if eventname == "FSDJump" and item.get("BodyType") == "Star":
                                    coordinates = item.get('StarPos', [])
                                    jump_info[system_name].update(dict(
                                        bodyid=item.get('BodyID'),
                                        bodies={
                                            item.get('BodyID'):dict(
                                                mainstar=True,
                                                scans=[]
                                            )},
                                        coord=coordinates,
                                    ))
                                    if coordinates:
                                        data[system_name] = coordinates
                                        systems.get(system_name)["coord"] = coordinates
                                        systems.get(system_name)['visits'] += 1

                                    # update last system
                                    if timestamp > last_system.get("timestamp"):
                                        last_system = {k:item.get(k, last_system.get(k)) for k in last_system }
                                        last_system['timestamp'] = timestamp
                                        
                                elif (eventname == "StartJump" and item.get("Jumptype") == "Hyperspace") or eventname == "FSDTarget":
                                    jump_info[system_name].update(dict(
                                        starclass=item.get('StarClass')
                                    ))
                                    
                                elif eventname == "FSSDiscoveryScan":
                                    jump_info[system_name].update(dict(
                                        bodycount=item.get('BodyCount')
                                    ))
                                    
                                elif eventname == "Scan":
                                    if item.get('BodyID') not in jump_info.get(system_name).get('bodies'):
                                        jump_info.get(system_name)['bodies'].update({
                                            item.get('BodyID'):dict(
                                                mainstar=False,
                                                scans=[])
                                        })
                                    body = jump_info.get(system_name).get('bodies').get(item.get('BodyID'))
                                    body['scans'].append(item.get("ScanType"))
                                    
                                    if "StarType" in item:
                                        body.update(dict(
                                            starclass=item.get("StarType"),
                                            subclass=item.get("Subclass"),
                                            stellarmass=item.get("StellarMass"),
                                            luminosity=item.get("Luminosity"),
                                            absolutemagnitude=item.get("AbsoluteMagnitude"),
                                        ))
                                        map_spectral(item.get('StarType'))
                                        map_luminosity(item.get('Luminosity'))
                                        map_magnitude(item.get('AbsoluteMagnitude'))
                                        
                                    elif "PlanetClass" in item:
                                        body.update(dict(
                                            planetclass=item.get("PlanetClass"),
                                            volcanism=item.get("Volcanism"),
                                            massem=item.get("MassEM"),
                                            radius=item.get("Radius"),
                                            terraformstate=item.get("TerraformState"),
                                            materials=item.get("Materials"),
                                        ))
                                    
                                    if item.get("ScanType")=="AutoScan":
                                        body.update(dict(
                                            distancefromarrival=item.get("DistanceFromArrivalLS"),
                                            wasdiscovered=item.get("WasDiscovered"),
                                            wasmapped=item.get("WasMapped"),
                                        ))
                                    elif item.get("ScanType")=="Detailed":
                                        body.update(dict(
                                            distancefromarrival=item.get("DistanceFromArrivalLS"),
                                            wasdiscovered=item.get("WasDiscovered"),
                                            wasmapped=item.get("WasMapped"),
                                        ))

                                elif eventname == 'Bounty':
                                    targetship = item.get('Target')
                                    if targetship not in bountyships:
                                        bountyships[targetship]=[]
                                    bountyships[targetship] += [V.get('Reward') for V in item.get('Rewards')]
                                    for voucher in item.get('Rewards'):
                                        bounty_faction = voucher.get("Faction")
                                        bounties[bounty_faction] = bounties.get(bounty_faction,0) + voucher.get('Reward')
                                        factions.get(bounty_faction,{}).update(
                                            bounty=max(timestamp,factions.get(bounty_faction, {}).get('bounty','')))


                                elif eventname == 'RedeemVoucher' and item.get('Type') == 'bounty':
                                    for voucher in item.get('Factions'):
                                        bounty_faction = voucher.get("Faction")
                                        vouchers[bounty_faction] = vouchers.get(bounty_faction, 0) + voucher.get('Amount')
                                        factions.get(bounty_faction,{}).update(
                                            bounty_voucher=max(timestamp,factions.get(bounty_faction,{}).get('bounty_voucher')))
                                

                            if eventname in scan_events:
                                body_name = item.get("BodyName")
                                if body_name not in bodies:
                                    bodies[body_name] = dict(IsPlanet=False, PlanetClass=None, Materials=dict())

                                body = bodies[body_name]
                                if 'bodies' not in system:
                                    system['bodies']={}
                                if system_name and body_name and body_name not in system.get('bodies',{}):
                                    system["bodies"][body_name] = body

                                for key in [
                                    "StarSystem", "DistanceFromArrivalLS", "BodyType", "PlanetClass", "SurfaceGravity",
                                    "TerraformState", "WasDiscovered", "WasMapped", "Landable", "ProbesUsed", "Signals",
                                    "Volcanism"
                                ]: #items
                                    if key in item:
                                        body[key] = item.get(key)

                                if "PlanetClass" in item:
                                    body["IsPlanet"] = True

                                signal = item.get("SignalName_Localised", None)
                                if signal and system:
                                    e = system.get("signals", set())
                                    e.add(signal)
                                    system["signals"] = e

                                materials = item.get("Materials")
                                if materials and system:
                                    body.get('Materials').update({M['Name']:M['Percent'] for M in materials})

                            if eventname == 'MultiSellExplorationData':
                                bodies_sold = sum([S.get('NumBodies',0) for S in item.get("Discovered",[])])
                                
                                if bodies_sold > 0:
                                    avg_body_val = item.get("TotalEarnings",0) / bodies_sold
                                else:
                                    avg_body_val = 0
                                for datum in item.get("Discovered",[]):
                                    sold_systems[datum.get('SystemName')] = datum.get('NumBodies') * avg_body_val
                                    
                                #for key in ["Signals", "Materials"]: # lists

                            

                        continue # -> while


                    #print(f"Empy chunk -> Done! Imported {system_count} systems in {round(time.process_time() - start,1)} seconds")
                    break

            #print(f"Updating Database ...")
            await push_query.executemany([[S] + data[S] for S in data])
            system_count += len(data)
            data = {}
            
        # ------------------------------
                        
    #print(f"{count}/{est_count}\t{system_count}\tsystems,\t{int(system_count / (time.process_time() - start))} /s,\t{round(100*count/est_count,2)}%, {round(((est_count - count) * (time.process_time() - start)/count),1)} remaining")


    if system_count > 0:
        tpl = (time.process_time() - start)/system_count
        print(f"\n{ (time.process_time() - start)} seconds {system_count} systems, per system {round(1000000*tpl,2)} us.") 
        
sessions = np.asarray(sessions[1:])
#print(f"Total playing time: {str(datetime.timedelta(seconds=np.sum(sessions[:,2])))}")

print(last_system)
system_name = last_system.get("StarSystem","Sol")

In [ ]:
sessions

In [ ]:
[f for f in factions.keys() if 'Blue Maf' in f]


In [ ]:
factions['Eurybia Blue Mafia']

In [ ]:
systems['Enayex']

## Mapping current system

In [ ]:
pd.DataFrame([
    (
        system_name, name.replace(system_name,''),round(body["DistanceFromArrivalLS"]),
        body.get("BodyType"),body.get("PlanetClass"), 
        len(body.get('Materials')), bool(body.get('TerraformState') == 'Terraformable'),
        body.get("WasDiscovered"),body.get("WasMapped"), body.get("ProbesUsed"),  
        round(planet_values.get(
            body.get("WasDiscovered"),{}).get(
            body.get("WasMapped"),{}).get(
            bool(body.get('TerraformState') == 'Terraformable'),{}).get(
            body.get("PlanetClass"),0)/1e6,1),
        body.get('Volcanism'),
        ",".join([f"{S['Count']} {S['Type_Localised']}" for S in body.get('Signals',[])]), 
        round(body.get('SurfaceGravity')/9.807,2)
    )  
    for name, body in systems[system_name]["bodies"].items() 
    #for S in body.get('Signals',[]) 
    if body["IsPlanet"] and (
        #(not body.get("WasDiscovered") and ('metal' in body.get("PlanetClass","").lower()) ) or
        planet_values.get(
            body.get("WasDiscovered"),{}).get(
            body.get("WasMapped"),{}).get(
            bool(body.get('TerraformState') == 'Terraformable'),{}).get(
            body.get("PlanetClass"),0) > 0
        #planet_values.get(body.get("PlanetClass"),False)
        or body.get('TerraformState') == 'Terraformable'
        #or 'life' in body.get("PlanetClass","").lower()
        #or body.get('Volcanism')
        or body.get("ProbesUsed") 
        #or (body.get('Landable', '') and body.get('Materials'))
    )
], columns=[
    "system","name","dist.(ls)","type","class", 
    'raw',"terra",'disc.',"mapped","probes", "value", "volcanism",
    'signal','gravity']).set_index(["system","name"]).sort_values(
    ["mapped","dist.(ls)"])
        
    
#if   not body.get("WasMapped") and not body.get("ProbesUsed")

## Signals

In [ ]:
pd.DataFrame([
    (
        N, 
        S.get('visits'),
        S.get('SystemFaction', '')  ,
        round(factions.get(S.get('SystemFaction'),{}).get('reputation',1)), 
        round(np.sqrt(np.sum(np.square(np.asarray(system.get('coord'))-np.asarray(S.get('coord'))))),1), 
        E.replace('Resource Extraction Site','RES').replace('Conflict Zone','CZ').replace(' Intensity]',']').replace('Unidentified signal source','USS')
    )
    for N, S in systems.items()
    for E in systems.get(N,{}).get("signals", set())
    if round(np.sqrt(np.sum(np.square(np.asarray(system.get('coord'))-np.asarray(S.get('coord'))))),1) < 60
    and ("Resource" in E )
    and not ("Hazardous" in E )

], columns=["system", "visits", 'faction',  'reputation', "distance", "signal"]).set_index(["system","faction"]).sort_values(['distance','signal']).head(30)

## Discoveries

In [ ]:
pd.DataFrame([
    (
        sname, bid ,
        body.get('starclass'),
        body.get("subclass"),
        body.get('wasdiscovered',systems.get(sname,{}).get('bodies',{}).get(sname,{}).get('WasDiscovered')),
        body.get('scans'),
        system.get('bodycount'),
        round(sold_systems.get(sname,0)/1e3)
        
    )
    for sname, system in jump_info.items()
    for bid, body in system["bodies"].items()
    if body.get("mainstar") and  not sold_systems.get(sname)
    #and not sold_systems.get(sname,0)
    #and systems.get(sname,{}).get('bodies',{}).get(sname,{}).get('WasDiscovered') is None
    #for S in body.get('Signals',[]) 
], columns=[
    "system","id","class", "subclass", 
    'disc.', 'scans', 'count', 'sold']).set_index(["system","id"]).sort_values(
    ["system","id"]).sort_values('sold')
        
    
#if   not body.get("WasMapped") and not body.get("ProbesUsed")

In [ ]:
systems['Thraikoo MR-N d6-154']['']

In [ ]:
jump_info['Flyoo Hypooe VZ-M d8-88']

## Features

In [ ]:
pd.DataFrame([
    [
        N, 
        # ----------------------- Logfiles
        I.get('bodycount',len(I.get('bodies', []))),
        len([i for i,b in I.get('bodies').items() if b.get('starclass') and not b.get('mainstar')]),   # star count
        round(sum([b.get('stellarmass') for i,b in I.get('bodies').items() if b.get('starclass')])),   # star mass
        str(I.get('bodies').get(I.get('bodyid')).get('starclass')) + str(I.get('bodies').get(I.get('bodyid')).get('subclass')), # spectral class
        I.get('bodies').get(I.get('bodyid')).get('luminosity'),
        I.get('bodies').get(I.get('bodyid')).get('absolutemagnitude'), 
        # ----------------------- EDSM
        round(I.get('edsm',{}).get('bodyCount',0)),
        len([1  for eb in I.get('edsm').get('bodies') if  eb.get('spectralClass') and not eb.get('isMainStar')]),
        round(sum([eb.get('solarMasses') for eb in I.get('edsm').get('bodies') if  eb.get('spectralClass')])) # solarMasses

    ] + [
        [eb for eb in I.get('edsm').get('bodies') if eb.get('isMainStar')][0].get(k) for k in ['spectralClass','luminosity','absoluteMagnitude'] 
    ] 
    for N, I in jump_info.items()
    
    if I.get('bodycount', 0) > 0 
    and I.get('bodies').get(I.get('bodyid')).get('luminosity') 
    and I.get('edsm',{}).get('bodies',[])
    and I.get('edsm',{}).get('bodyCount')
    
]).set_index(0).sample(20)


In [ ]:
edsm_info_cache.get('Traikeou FN-P c7-0')

In [ ]:
[i for i,b in jump_info['Blau Thua NX-T d3-5'].get('bodies').items() if b.get('starclass') and not b.get('mainstar')]


In [ ]:
[(i,b) for i,b in jump_info['Blau Thua QD-S d4-4'].get('bodies').items() if b.get('starclass')]


In [ ]:
jump_info['Blau Thua NX-T d3-5']

In [ ]:
edsm_info_cache.get('Nyeajeou VK-D d13-48')

# Routing V2

## Targets

In [ ]:
importlib.reload(edcompanion.edsm_api)
from edcompanion.edsm_api import get_edsm_info

In [ ]:
importlib.reload(edcompanion.edsm_api)
importlib.reload(follow_log)
from edcompanion.edsm_api import get_edsm_info
from follow_log import follow_journal

#systems, system_name = follow_journal(verbose=False)

### Guardian

In [ ]:
await find_system(system_name)

In [ ]:
for reportname in "grreports gsreports gbreports".split(' '):
    df = pd.read_csv('data/grreports.csv')["systemName coordX coordY coordZ bodyName reportComment".split(" ")]
    


In [ ]:
for j in [row.reportComment
            for reportname in "grreports gsreports gbreports".split(' ') 
            for row in pd.read_csv(f'data/{reportname}.csv')["systemName coordX coordY coordZ bodyName reportComment".split(" ")].itertuples()
            if isinstance(row.reportComment, str) and ',' in row.reportComment
            
        ]:

        print(json.loads(j))
        break


In [ ]:
j

In [ ]:
j

In [ ]:
dict(markers=[
    dict(
        pin='cyan',
        text=f"{n:16} {s}",

    )
])

In [ ]:
pincolors=dict(
    grreports='cyan',
    gsreports='green',
    gbreports='yellow'
)
with open('custom/guardian_report.json', 'wt') as of:
    json.dump(dict(markers=[
        dict(
            pin=pincolors[reportname],
            text=f"{row.systemName:24} {row.bodyName:32}\n{json.loads(row.reportComment).get('Name_Localised') if (isinstance(row.reportComment, str) and ',' in row.reportComment) else ''}",
            **{k:v for k,v in dict(await find_system(row.systemName)).items() if k in ['x','y','z']}
        )
        for reportname in "grreports gsreports gbreports".split(' ') 
        for row in pd.read_csv(f'data/{reportname}.csv')["systemName coordX coordY coordZ bodyName reportComment".split(" ")].itertuples()
        
    ]), of)



In [ ]:
math

In [ ]:
report_guardianruins = pd.read_csv('data/grreports.csv')["systemName coordX coordY coordZ bodyName reportComment".split(" ")]

In [ ]:
pd.read_csv('data/gbreports.csv')

In [ ]:
report_guardiansites

In [ ]:
with open('custom/guardian_report.json', 'wt') as of:
    json.dump(dict(markers=[
        dict(
            pin='purple',
            text=f"{n:16} {s}",
            **{k:v for k,v in dict(await find_system(s)).items() if k in ['x','y','z']}
        )
        for n,s in targetsystems.items()
    ]), of)

### POI data

In [ ]:
poi_data = {}


In [ ]:
import csv

DVRzMdG_poilist = []

map_DVRitem = dict(
    coordinates=lambda I: np.asarray([float(I.get(ik, np.nan).replace(',','.')) for ik in ['x','y','z']]),
    name=lambda I: ' '.join(I.get('notiz').split('\n')[0].split(' ')[4:-1]),
    region=lambda I: '',
    type=lambda I: '',
    summary=lambda I: I.get('notiz'),
    curation=lambda I: 9,

)

for csvfile in sorted(
        [os.path.join('data', f) for f in os.listdir('data') if 'DVRzMdG' in f.split('_')[0] and '.csv' in f],
        key=lambda f:f.lower()
    ):
    print(csvfile)
    with open(csvfile, newline='', encoding='utf-8') as f:
        header, *datarows = [i for i in csv.reader(f,delimiter=';', quotechar='"', quoting=csv.QUOTE_MINIMAL)]
        DVRzMdG_poilist += [
            { k.lower():v for k, v in zip(header, r) if k }
            for r in datarows
        ]
        
        poi_data.update({
            item.get('systemname'):{
                c:map_DVRitem.get(c)(item)
                for c in ['coordinates','name','region','type','summary','curation']
            }
            for item in DVRzMdG_poilist
            if item.get('systemname')
                 and all([True if item.get(ik) else False for ik in ['x','y','z'] ])
        })    

In [ ]:
        
map_carrieritem = dict(
    coordinates=lambda I: np.asarray([float(I.get(f"coord_{ik}", np.nan).replace(',','.')) for ik in ['x','y','z']]),
    name=lambda I: f"{I.get('name')}",    
    callsign=lambda I:f"{I.get('callsign')}",
    region=lambda I: I.get('estimatedregion'),
    owner=lambda I: I.get('owner'),
    services=lambda I: set(I.get('services').split(';') if I.get('services') else [])
)
carrier_list=[]
with open('data/fleetcarriers.csv', newline='', encoding='utf-8') as f:
    header, *datarows = [i for i in csv.reader(f,delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)]
    carrier_list += [
        { k.lower():v for k, v in zip(header, r) if k }
        for r in datarows
    ]

carrier_data ={}
for item in carrier_list:
    if all([True if item.get(f"coord_{ik}") else False for ik in ['x','y','z'] ]) and item.get('services') and 'DSSA' in item.get('name') :
        if item.get('lastsystem') not in carrier_data:
            carrier_data[item.get('lastsystem')]=dict(
                coordinates=map_carrieritem.get('coordinates')(item),
                services=set(['']),
                carriers=[]
            )
        carrier_data.get(item.get('lastsystem')).get('carriers').append({
                map_carrieritem.get('callsign')(item):map_carrieritem.get('name')(item)
            }            
        )
        carrier_data.get(item.get('lastsystem')).get('services').update( map_carrieritem['services'](item))


In [ ]:
async with aiohttp.ClientSession() as session:
    async with session.get('https://edastro.com/gec/json/all') as req:
        _edastro_poi = await req.json()
        poi_data.update({
            item.get('galMapSearch'):{
                c:item.get(k)
                for c,k in zip(['coordinates','name','region','type','summary','curation'], ['coordinates','name','region','type','summary','curation'])
            }
            for item in _edastro_poi
        })

In [ ]:
pd.DataFrame.from_dict(poi_data)

In [ ]:
poi_systems = np.asarray([ 
    i.get('coordinates')
    for k, i in poi_data.items() ])
poi_systemnames = np.asarray([ 
    (k, i.get('name'),  i.get('summary'), i.get('curation'))
    for k, i in poi_data.items() ])

In [ ]:
def find_nearest_poi(coord1, coord2=None):
    p1 = np.asarray(coord1)
    p2 = np.asarray(coord2) if coord2 is not None else coord1

    poi_distances = np.linalg.norm(poi_systems[:, [0,1,2]] - p1, axis=1) + np.linalg.norm(poi_systems[:, [0,1,2]] - p2, axis=1)
    ordering = poi_distances.argsort()
    return poi_systems[ordering][0], poi_systemnames[ordering][0]

def find_poi_between(coord1, coord2):
    p1 = np.asarray(coord1)
    p2 = np.asarray(coord2)
    
    distance = np.sqrt(np.sum(np.square(p1-p2)))
    
    poi_distances1 = np.linalg.norm(poi_systems[:, [0,1,2]] - p1, axis=1) 
    poi_distances2 = np.linalg.norm(poi_systems[:, [0,1,2]] - p2, axis=1)
    
    poi_n = (
                np.less(poi_distances1, distance) &
                np.less(poi_distances2, distance)
            )
    return poi_systems[poi_n], poi_systemnames[poi_n], np.argmin(poi_distances1[poi_n])
    

    

In [ ]:
targetsystems = {L.split('\t')[0].replace('#','Waypoint '):L.split('\t')[1].split('(')[0].strip() for L in '''
#1	HIP 117029		
0.00%
0.00%
#2	Drojia YW-B d13-4	4,377.94 ly	
0.00%
0.00%
#3	Lysooh WT-R b7-0 (Halley's World)	7,658.41 ly	
0.00%
0.00%
#4	Plaa Ain FF-Z d76 (Rekohu Project)	10,162.39 ly	
0.00%
0.00%
#5	Blaa Hypai LA-J c11-3	13,299.43 ly	
0.00%
0.00%
#6	Floawns XE-R d4-45 (The Three Kings)	15,978.96 ly	
0.00%
0.00%
#7	Mynoaw LC-L d8-1429	21,214.37 ly	
0.00%
0.00%
#8	Egnairs AA-A h72 (Mairon)	26,508.32 ly	
0.00%
0.00%
#9	Hypiae Aurb AA-A g588 (Planet Pancake)	29,889.18 ly	
0.00%
0.00%
#10	Juenae XZ-G d10-651 (Red River Run)	34,771.56 ly	
0.00%
0.00%
#11	Sagittarius A*	36,288.63 ly	
0.00%
0.00%
#12	Hypio Proo VE-Q e5-1485	38,824.33 ly	
0.00%
0.00%
#13	Pho Aoscs OS-U f2-26 (Black Fields)	43,891.03 ly	
0.00%
0.00%
#14	Athaip WR-H d11-7577	48,599.27 ly	
0.00%
0.00%
#15	Dryau Scraa AA-A h747 (The Shiner)	54,319.41 ly	
0.00%
0.00%
#16	Chroabs TI-S d4-58 (Goliath)	58,392.41 ly	
0.00%
0.00%blu
#17	Ellairb SJ-B b42-10 (Lair of Unicorns)	63,416.44 ly	
0.00%
0.00%
#18	Plaa Aescs BK-Q d5-60	66,359.12 ly	
0.00%
0.00%
#19	Bleia Dryiae XJ-R e4-1	69,286.00 ly	
0.00%
0.00%
#20	Ocshodhis	74,359.19 ly	
0.00%
0.00%
'''.splitlines() if '#' in L}

In [ ]:
with open('custom/reise_markers.json', 'wt') as of:
    json.dump(dict(markers=[
        dict(
            pin='purple',
            text=f"{n:16} {s}",
            **{k:v for k,v in dict(await find_system(s)).items() if k in ['x','y','z']}
        )
        for n,s in targetsystems.items()
    ]), of)

In [ ]:
## interesting systems
targetsystems.update(dict(
    sol='Sol',
    ix='Ix',
    proto_1='Dehoae HH-U e3-14',
    guardian_fsd1='HD 63154',
    guardian_fsd2='Synuefe PX-J c25-8',
    mel_brandon='Luchtaine',
    marsha_hicks='Tir',
    # Verruckte POI
    three_dwarfs='Drojia YW-B d13-4',
    eye_of_fatima='Drojaea DG-E d12-4',
    gravi_nightmare='Drojia XK-D c26-0',
    # Events
    hip22460='HIP 22460',
    running_man='HIP 23759',
    fc_hip22460='Pleiades Sector CW-U b3-2',
    uia_3='Synuefai XI-F c2', #[-243,-170,-1033],
    # Asteroid & deep space bases
    witch_science='HIP 23759',
    orion_tourist='PMD2009 48',
    medusa='Crescent Sector GW-W c1-8',
    anchorage='Rohini',
    new_growth='Pencil Sector EL-Y d5',
    # Nebula / POI
    scorch_red_moon='Blue Hypooe VV-A c2-12',
    cygnus_x1='V1357 Cygni',
    elephant='IC 1396 Sector QI-S d4-9',
    ngc7026='Csi+47-21046',
    ngc7354='Csi+61-22385',
    heartsoul='Hypoae Ain MO-I d9-37',
    siteseeing_01='Phua Bre FB-O e6-257',
    thors_eye="Thor's Eye",
    sagitarius_a='Sagittarius A*',
    annihilator='Great Annihilator',
    four_of_a_kind='Dryao Phylio AA-A h410',
    witch_head='Witch Head Sector BQ-Y d14',
    amundsen='Lyed YJ-I d9-0',
    five_eyes='Phrio Phoea AA-A h12',
    statue_liberty='Statue of Liberty Sector DL-Y d27',
    # Carriers
    sublime='Gleeque HW-N e6-149',
    artemis='Synuefuae CM-J d10-42',
    gorgon='NGC 7822 Sector BQ-Y d12',
    rocksteady='Prooe Hypue FH-U e3-2',
    paradox='Prai Hypoo TX-B d4', # carrier
    inverness='Thraikoo PS-U e2-4',       # carrier
    maerzenbecher='Hedgo GL-P c5-4',      # carrier
    fuelmoon='Hypo Aeb WK-H b51-0',
    brazillian='Graesms CW-E d11-2',
    sanctuary='Syreadiae JX-F c0',        # carrier
    ngc281='BD+55 191',
    farsight='Heart Sector IR-V b2-0',
    minbari="Eor Aescs UP-N c20-0",
    traikeou='Traikeou WL-I c11-1',
    nearby_station='Prielo TZ-G d10-4',
    # Voyager journey
    caretaker='Byoo Briae TJ-G c24-0',
))

### Verruckte Reise

In [ ]:
vri = 14

In [ ]:
vri+=1

In [ ]:
start_system = await find_system(targetsystems.get(f'Waypoint {vri}'))
target_system = await find_system(targetsystems.get(f'Waypoint {vri+1}'))
end = np.round(np.asarray([target_system.get(k) for k in ["x","y","z"]]))
print(f"waypoint {vri:2}: {dict(start_system)}")
print(f"waypoint {vri+1:2}: {dict(target_system)}")

In [ ]:
vri

### Various

In [ ]:
print(await find_system(targetsystems.get('eye_of_fatima')))

In [ ]:
print( get_edsm_info(targetsystems.get('eye_of_fatima'), False))

In [ ]:
# Setup target system vars
target_system = await find_system(targetsystems.get('eye_of_fatima'))
if target_system:
    end = np.round(np.asarray([target_system.get(k) for k in ["x","y","z"]]))
else:
    target_system = get_edsm_info(targetsystems.get('eye_of_fatima'), False)
    end = np.round(np.asarray([target_system.get('coords').get(k) for k in ["x","y","z"]]))

In [ ]:
# Setup target system vars
target_system = await find_system(targetsystems.get(f'DRMG{vri}'))
end = np.round(np.asarray([target_system.get(k) for k in ["x","y","z"]]))

In [ ]:
# Setup target system vars
target_system = await find_system(targetsystems.get('paradox'))
end = np.round(np.asarray([target_system.get(k) for k in ["x","y","z"]]))

In [ ]:
dict(target_system)

In [ ]:
# Setup target system vars
target_system = await find_system(targetsystems.get('running_man'))
end = np.round(np.asarray([target_system.get(k) for k in ["x","y","z"]]))

In [ ]:
# Setup target system vars
target_system = await find_system(targetsystems.get('statue_liberty'))
end = np.round(np.asarray([target_system.get(k) for k in ["x","y","z"]]))

In [ ]:
# Setup target system vars
target_system = await find_system('Eurybia')
end = np.round(np.asarray([target_system.get(k) for k in ["x","y","z"]]))
print(end)

## Startpoint

In [ ]:
# Set starting point vars and initialize route & path vars
start_system = await find_system('Ix')

In [ ]:
# Set starting point vars and initialize route & path vars
start_system = await find_system(N)

In [ ]:
# Set starting point vars and initialize route & path vars
start_system = await find_system(route[-1][2].rstrip())

In [ ]:
from follow_log import follow_journal
systems, system_name = follow_journal(verbose=False)

In [ ]:
system_name = get_commander_position('immerlicht', os.getenv('EDSM_TOKEN')).get('system')

In [ ]:
# Set starting point vars and initialize route & path vars
start_system = await find_system(system_name,100)

## Calculate route waypoints

In [ ]:
print(start_system)
print(target_system)
#start_system = await find_system('Sol')
start = np.round(np.asarray([start_system.get(k) for k in ["x","y","z"]]))

path = [start] # contains coordiantes of waypoints
system_names = [(await find_system(start.tolist())).get('name')] # systemnames of waypoints
path_info = {} # information about waypoints and algorithm execution

visit_poi = True
cube_side = 80
search_side = 1200



In [ ]:
# Add no mare then x waypoints
pause_at = len(path)+11

# distance from current path end to final target
travel_distance = np.sqrt(np.sum(np.square(end-path[-1])))
print(system_names[-1], path[-1])
print(f"\t{round(travel_distance)} ly -> {(await find_system(end.tolist())).get('name')} {np.round(end).tolist()}")

# open DB connection
async with pgpool.acquire() as pgconnection:
    
    while cube_side > 20 and len(path) < pause_at:
        # Fix cube_side to be divisible by two (easier on our center points, though not neccesary)
        cube_side = round(2 * math.floor(cube_side/2))
        wxz = 16 * cube_side
        search_side = max(search_side, cube_side*9)
        cube_radius = np.sqrt(np.sum(np.square([cube_side/2, cube_side/2, cube_side/2])))
        def cube_center(point):
            return [round(cube_side*math.floor(v/cube_side) + cube_side/2) for v in point]
        def cube_location(point):
            return [round(cube_side*math.floor(v/cube_side) + cube_side/2) for v in point]

        # End of search condition 
        while travel_distance > 500 and len(path) < pause_at:
            currentwp = cube_center(np.round(path[-1],2))

            travel_distance = np.sqrt(np.sum(np.square(end - currentwp)))
            direction = (end - currentwp)/travel_distance

            # first estimated waypoint 
            wpdistance = math.floor(0.9*search_side/cube_side) * (
                cube_side if len(path) > 2 and travel_distance > search_side else 2*cube_side)
            nextwp = cube_center(np.round(currentwp + min(wpdistance, travel_distance) * direction,2))
            sys.stdout.write(f"\nCurrent {str(currentwp)} to {str(nextwp)}")

            poiinfo = None
                
            poi_distances1 = np.linalg.norm(poi_systems[:, [0,1,2]] - currentwp, axis=1) 
            poi_distances2 = np.linalg.norm(poi_systems[:, [0,1,2]] - nextwp, axis=1)
            poi_n = np.less(poi_distances1, wpdistance) & np.less(poi_distances2, wpdistance)
            poi_targets = poi_systems[poi_n]
            sys.stdout.write(f" {poi_targets.shape[0]} POI's")

            if visit_poi and poi_targets.shape[0]>0 and target_system.get('name') not in poi_data:
                nextpoiidx = np.argmin(poi_distances1[poi_n])
                nextwp = cube_center(poi_systems[poi_n][nextpoiidx])
                sys.stdout.write(f" set nextwp: {nextwp}")
                poiinfo = [dict(system=r[0],name=r[1], description=r[2], coord=c) for r,c in zip(poi_systemnames[poi_n],poi_systems[poi_n])][nextpoiidx]
                sys.stdout.write(f" POI: {poiinfo['system']} {poiinfo['name']}")
                        
            
            location_cube = [round(cube_side*math.floor(v/cube_side)) for v in currentwp]
            destination_cube = [round(cube_side*math.floor(v/cube_side)) for v in nextwp]
            wxz = max(2*cube_side,min(wxz, cube_side*round(travel_distance/(6*cube_side)))) 
            if np.square(direction[0]) > np.square(direction[2]): # travel more along x then z axis
                enclosure = [
                    min(location_cube[0], destination_cube[0]) - cube_side, max(location_cube[0], destination_cube[0]) + cube_side + 1,
                    min(-1500, location_cube[0], destination_cube[1]), max(1500, location_cube[0], destination_cube[1]),
                    min(location_cube[2], destination_cube[2]) - wxz, max(location_cube[2], destination_cube[2]) + wxz+1,
                ]    
            else:
                enclosure = [
                    min(location_cube[0], destination_cube[0]) - wxz, max(location_cube[0], destination_cube[0]) + wxz+1,
                    min(-1500, location_cube[0], destination_cube[1]), max(1500, location_cube[0], destination_cube[1]),
                    min(location_cube[2], destination_cube[2]) - 1*cube_side, max(location_cube[2], destination_cube[2]) + 1*cube_side + 1,
                ]    

            t = [int(x) for x in enclosure]
            extend = [x for x in zip(t[::2], t[1::2])]  
            
            # Note: from now work with the center-points of the cubes
            full = np.unique(np.asarray([(round(cx + cube_side/2),round(cy + cube_side/2),round(cz + cube_side/2))  
                                             for cx in range(*extend[0], cube_side) 
                                             for cy in range(*extend[1], cube_side) 
                                             for cz in range(*extend[2], cube_side)], 
                                        dtype=[("cx","int64"),("cy","int64"),("cz","int64")]))

            sys.stdout.write(f"\rCurrent {str(currentwp)} to {str(nextwp)}, {str(full.shape[0])} cubes ...  {poi_targets.shape[0]} POI's")
            
            candidate_cubes = await pgpool.fetch('''
                SELECT ROUND($7*FLOOR(x/$7) + $7/2) AS cx, ROUND($7*FLOOR(y/$7) + $7/2) AS cy, ROUND($7*FLOOR(z/$7) + $7/2) AS cz, 
                        count(1)  filter(where not n) as starcount, 
                        ROUND(|/((AVG(x)-$8)^2 + (AVG(y)-$9)^2 + (AVG(z)-$10)^2)) distance,
                        0 weight,
                        count(*) filter(where n) as ncount
                FROM systems 
                WHERE x >= $1 AND x <= $2 AND  y >= $3 AND y <= $4  AND z  >= $5 AND z <= $6 
                GROUP BY cx, cy, cz

            ''', *enclosure, cube_side, *end.tolist())

            regions = np.unique(np.asarray(
                [tuple(R) for R in candidate_cubes], 
                dtype=[("cx","int64"), ("cy","int64"),("cz","int64"),("starcount","float64"), ("distance","float64"), ("weight","float64"),("ncount","float64")]))

            sys.stdout.write(f"\rCurrent {str(currentwp)} to {str(nextwp)}, {str(regions.shape[0])} regions found ...")
            assert not regions.shape[0] > full.shape[0]
            joined = join_by(
                ('cx', 'cy','cz'), 
                full,regions, 
                jointype="outer", usemask=False,
                defaults = {"starcount":0, "total":np.nan, "distance":np.nan, 'weight':np.nan, 'ncount':0}
            )

            joined["distance"] = np.round(np.sqrt(
                np.square(joined["cx"]-end[0]) + 
                np.square(joined["cy"]-end[1]) + 
                np.square(joined["cz"]-end[2]))) + np.round(np.sqrt(
                np.square(joined["cx"]-currentwp[0]) + 
                np.square(joined["cy"]-currentwp[1]) + 
                np.square(joined["cz"]-currentwp[2])))

            joined['weight'] = np.log(1 + (1+joined['starcount'])/(1+joined['ncount']))
            #joined['weight'] = np.round(np.log((3+joined['starcount'])/(1+joined['ncount'])) * joined["distance"]/travel_distance)
            #joined['weight'] = np.round(np.log((3+joined['starcount'])/(1+joined['ncount'])) * joined["distance"])
            #max_count = np.amax(joined['starcount'])

            # Travel direction, z-x order and normal
            if np.square(direction[0]) > np.square(direction[2]): # travel more along x then z axis
                main_direction = direction[0]
                main_order = ['cx', 'cz']
                plane_normal = np.array([1,0,0])

            else: # travel more along z then x axis
                main_direction = direction[2]
                main_order = ['cz','cx']
                plane_normal = np.array([0,0,1])

            if main_direction < 0:
                joined.sort(order=main_order)
                plane_normal *= -1
            else:
                joined[::-1].sort(order=main_order) # reverse sort

            #('cx', 'cy', 'cz', 'starcount', 'distance', 'weight', 'ncount')
            cubespace = np.zeros(shape=(joined.shape[0],len(joined.dtype)+1+2))
            for ci, cn in zip(range(len(joined.dtype)), joined.dtype.names) :
                cubespace[:,ci] = joined[cn]

            candidate_waypoints = joined[np.less(joined['distance'], np.percentile(joined['distance'],8))]
            waypoints = np.zeros(shape=(candidate_waypoints.shape[0],7))
            waypoints[:,3] = candidate_waypoints['starcount'] * candidate_waypoints['weight'] 
            for ci, cn in zip([0,1,2], ['cx', 'cy','cz']) :
                waypoints[:,ci] = candidate_waypoints[cn]

            # calculate a weight for each line from currentwp to waypoints
            # based on the distance of each cube in our extend and the starcount
            #sys.stdout.write(f", {waypoints.shape[0]} waypoints")
            for waypoint in waypoints:
                # First calculate distances between cubes and travel 'lines'
                # d = norm(np.cross(lp2-lp1, lp1-p3))/norm(lp2-lp1)
                lp1 = waypoint[0:3]
                lp2 = currentwp
                p3 = cubespace[:,0:3]
                
                d = np.linalg.norm(np.cross(lp2-lp1, lp1-p3,axisb=1),axis=1)/max(np.linalg.norm(lp2-lp1),0.1)

                w = np.less(d,4*cube_radius) # wider selection to get a relative density
                n = np.less(d,2*cube_radius) # select cubes near the travel line
                # sum and divide by traveldistance to get a measure of density
                waypoint[4] = np.sum(cubespace[n][:,3]/(1+d[n])) / (1+np.sqrt(np.sum(np.square(lp1-lp2))))
                waypoint[5] = np.mean(cubespace[n][:,3])
                waypoint[6] = np.mean(cubespace[n][:,6])
                waypoint[4] *= (1+np.mean(cubespace[n][:,3])) / (1+np.mean(cubespace[w][:,3]))

            sys.stdout.write(f"\rCurrent {str(currentwp)} to {str(nextwp)}, weighed {str(waypoints.shape[0])} waypoint cubes\r")
            path_info[system_names[-1]] = {'candidates':[], 'stations':[]}
            waypoints = waypoints[np.less(waypoints[:,4], np.percentile(waypoints[:,4],15))]
            
            if candidate_cubes:
                candidates = []

                for weighed_target in waypoints[waypoints[:,4].argsort()]:
                    sys.stdout.write(f"\rTarget, w={weighed_target[4]}: {np.round([weighed_target[k] for k in [0,1,2]])}\r")
                    s = cube_side
                    while not candidates and s < 200:
                        s += cube_side
                        target_coordinates = np.asarray([weighed_target[k] for k in [0,1,2]])
                        if poiinfo:
                            if 4*s > np.sqrt(np.sum(np.square(target_coordinates - poiinfo['coord']))):
                                candidates = await find_nearby_systems(poiinfo['coord'],2*s, include_neutron=False)
                                sys.stdout.write(f"... Found {len(candidates)} POI for search\r")
                            else:
                                sys.stdout.write(f" ... POI distance {np.sqrt(np.sum(np.square(target_coordinates - poiinfo['coord'])))} \r")
                            
                        if not candidates:
                            candidates = await find_nearby_systems(target_coordinates, s, include_neutron=False)
                            
                        sys.stdout.write(f"\rFound {len(candidates)} candidates for {np.round([weighed_target[k] for k in [0,1,2]])} {s} ly search\r")

                    if candidates:   
                        for candidate in candidates: 
                            #candidate = candidates[0]
                            #print(candidate)
                            path_info[system_names[-1]]['candidates'].append(record_to_dict(candidate))
                            sys.stdout.write(f"\rFound candidate {record_to_dict(candidate).get('name')} for {np.round([weighed_target[k] for k in [0,1,2]])} {s} ly search\r")
                            #path_info[system_names[-1]]['stations'] += [(R.get('system'),R.get('station'), round(R.get('distance'))) for R in await find_nearby_stations([joined[0][k] for k in ["cx","cy","cz"]],300)]
                            if candidate.get("name") in system_names:
                                sys.stdout.write(f" ... is already a waypoint")
                                continue

                            if np.round(np.sqrt(np.sum(np.square(path[-1]-end)))) < 2*cube_side+np.round(np.sqrt(np.sum(np.square(np.asarray([candidate.get(k) for k in ["x","y","z"]])-end)))):
                                sys.stdout.write(f" {path[-1]} is already closer to {end} than {[round(candidate.get(k)) for k in ['x','y','z']]} {candidate.get('name')}")
                                continue

                            cube_density = weighed_target[3]
                            path.append(np.asarray([candidate.get(k) for k in ["x","y","z"]]))
                            sys.stdout.write(f"""\r{system_names[-1]:26} {np.sqrt(np.sum(np.square(path[-1]-currentwp))):7.1f} ly ->\t{candidate.get('name'):26} {weighed_target[4]:6.2f}\t{np.sqrt(np.sum(np.square(path[-1]-end))):7.1f} ly remaining {' ':12}""")

                            path_info[system_names[-1]]['nextwp']=dict(
                                system=candidate.get('name'),
                                distance=np.round(np.sqrt(np.sum(np.square(path[-1]-currentwp))),1),
                                extend=extend,
                                wxz=wxz,
                                remaining=np.round(np.sqrt(np.sum(np.square(path[-1]-end))),1),
                                weight=weighed_target[4],
                                density=1000*weighed_target[5]/math.pow(cube_side,3),#1e6*weighed_target[3]/(np.power(cube_side/3.26,3)*weighed_target[6]),
                                ndensity=1000*weighed_target[6]/math.pow(cube_side,3),#1e6*weighed_target[3]/(np.power(cube_side/3.26,3)*weighed_target[6]),
                                cube=[weighed_target[k] for k in [0,1,2]],
                                error=np.round(np.sqrt(np.sum(np.square(path[-1]-np.asarray(nextwp))))),
                                poi=[dict(system=r[0],name=r[1], description=r[2], coord=c) for r,c in zip(poi_systemnames[poi_n],poi_systems[poi_n])]
                            )
                            system_names.append(candidate.get("name"))
                            break
                        break

                if np.sqrt(np.sum(np.square(np.asarray(cube_center(np.round(path[-1],2))) - currentwp))) < cube_side:
                    break

            else:
                sys.stdout.write(f"\rCurrent {str(currentwp)} to {str(nextwp)}, no candidate-cubes\r")
                break

        cube_side = int(0.8*cube_side)
        
    print()    
    with open('custom/route.json', 'wt') as of:
        json.dump(dict(markers=[
            dict(
                pin='red',
                text=f"{I.get('system')}\n{str(I.get('cube'))}\n{str(I.get('extend'))}",
                x=I.get('cube',[])[0],
                y=I.get('cube',[])[1],
                z=I.get('cube',[])[2],
                #**{k:v for k,v in dict(await find_system()).items() if k in ['x','y','z']}
            )
            for W,I in {WP:N.get('nextwp',{}) for WP, N in path_info.items()}.items() 
            if I.get('cube')
            #for n,s in targetsystems.items()
        ]), of,indent=4, sort_keys=False)

In [ ]:
pd.DataFrame([
    (
        W,I.get('distance'), I.get('system'), I.get('remaining'), 
        I.get('error'),1000*I.get('weight',0),I.get('density'),I.get('ndensity',1)/I.get('density',1),
        I.get('cube',[np.nan,np.nan,np.nan])[0],
        I.get('cube',[np.nan,np.nan,np.nan])[1],
        I.get('cube',[np.nan,np.nan,np.nan])[2],
        I.get('extend'),
        ' | '.join([f"{p.get('system')} - {p.get('name')}" for p in I.get('poi',[])])
    ) 
    for W,I in {WP:N.get('nextwp',{}) for WP, N in path_info.items()}.items()],
    columns=['waypoint', 'distance', 'nextwp', 'remaining','stray','weight', 'density','neutron','x','y','z','extend', 'poi']
).set_index('waypoint').dropna()#.to_markdown(f'waypoints-{system_names[0]}-{system_names[-1]}.md')

In [ ]:
pd.DataFrame(joined).mean(axis=0)

## Nav Route

In [ ]:
importlib.reload(edcompanion.edsm_api)
importlib.reload(follow_log)
from edcompanion.edsm_api import get_edsm_info
from follow_log import follow_journal
system_name = get_commander_position('immerlicht', os.getenv('EDSM_TOKEN')).get('system')
#systems, system_name = follow_journal(verbose=False)

In [ ]:
neutron_db = {}
with open('collected_neutron_systems.json', "rt") as jsonfile:
    neutron_db=json.load(jsonfile)

await pgpool.executemany(
    """INSERT INTO eddb.systems (name, x, y, z, n) 
        VALUES ($1, $2, $3, $4,TRUE) 
        ON CONFLICT (name) DO UPDATE SET n=TRUE
    """, [(
    s.get('StarSystem'),        
    s.get('StarPos')[0],
    s.get('StarPos')[1],
    s.get('StarPos')[2]

) for n, s in neutron_db.items() if 'N' == s.get('StarClass')])


In [ ]:
#logpath = "/Users/fenke/Saved Games/Frontier Developments/Elite Dangerous"
navfile = os.path.join(logpath, "NavRoute.json")
print(navfile)

route = []
edsm_info={}

async with aiohttp.ClientSession() as session:

    count = 0

    with open(navfile, "rt") as jsonfile:
        navroute = json.load(jsonfile)
        #prettyprint(navroute)

        for item in navroute.get('Route'):
            N = item.get('StarSystem')
            #print(N, *[c for c in item.get('StarPos')])
            #assert False
            await pgpool.execute(
                """INSERT INTO eddb.systems (name, x, y, z) 
                    VALUES ($1, $2, $3, $4) 
                    ON CONFLICT DO NOTHING
                """,
                N, *[c for c in item.get('StarPos')]
            )

            edsm_info = get_edsm_info(N)
            route.append([
                '*' if system_name==N else ">" if not edsm_info else " " ,
                "#" if item.get('StarClass') in ['A','F','G'] else " ",
                f"{edsm_info.get('name'):26}" if edsm_info else f"{item.get('StarSystem'):26}",
                f"{systems.get(N,item).get('StarClass',([B.get('subType').replace('Star', '') for B in edsm_info.get('bodies',[{}]) if B.get('isMainStar')]+[''])[0])}", 
                f"{len([B.get('discovery') for B in edsm_info.get('bodies',[])])} / {edsm_info.get('bodyCount',len(systems.get(N,{}).get('bodies',{})))}",
                np.round(np.sqrt(np.sum(np.square(np.asarray([c for c in item.get('StarPos')])-end))),1),
                
                ", ".join([f"{round(c):6}" for c in item.get('StarPos')]),
                "".join([B.get('discovery',{}).get('commander','') for B in edsm_info.get('bodies',[{}]) if B.get('isMainStar') and B.get('discovery')]), 

            ])


with pd.option_context('display.max_rows', len(route)+1, 'display.max_columns', len(route[0])+1):
    display(
        pd.DataFrame(
            route, 
            columns=["d", "s", "system","class", "bodies", "remaining", "starpos", "commander"])   
    
    ) #need display to show the dataframe when using with in jupyter
    #some pandas stuff

## Thin-routes

In [ ]:
importlib.reload(edcompanion.edsm_api)
from edcompanion.edsm_api import get_edsm_info, get_systems_in_cube, get_systems_in_sphere, distance_between_systems

In [ ]:
target_name='Pho Aoscs OS-U f2-26'

target = dict(name=get_edsm_info(target_name,False).get('name'))
target.update(dict(coords=np.asarray([get_edsm_info(target_name,False).get('coords').get(k) for k in ['x','y','z']])))

In [ ]:
current_name='Pho Aoscs OY-Z d13-10'

current = dict(name=get_edsm_info(current_name,False).get('name'))
current.update(dict(coords=np.asarray([get_edsm_info(current_name,False).get('coords').get(k) for k in ['x','y','z']])))

In [ ]:
current=dict(
    name=last_system.get('StarSystem'),
    coords=np.asarray(last_system.get('StarPos'))
)

In [ ]:
distance=np.ceil(np.sqrt(np.sum(np.square(current.get('coords')-target.get('coords')))))
distance

In [ ]:
nearby = get_systems_in_cube(target['name'],100)
nearby.sort(key = lambda S:S['distance'])

In [ ]:
nearby = get_systems_in_cube('Pho Aoscs OS-U f2-44',100)
nearby.sort(key = lambda S:S['distance'])

In [ ]:
nearby = get_systems_in_cube('Pho Aoscs BQ-P e5-4',100)
nearby.sort(key = lambda S:S['distance'])

In [ ]:
nearby = get_systems_in_cube('Pho Aoscs FW-N e6-7',100)
nearby.sort(key = lambda S:S['distance'])

In [ ]:
nearby = get_systems_in_cube('Pho Aoscs FW-N e6-5',100)
nearby.sort(key = lambda S:S['distance'])

In [ ]:
nearby = get_systems_in_cube('Pho Aoscs LS-B d13-16',100)
nearby.sort(key = lambda S:S['distance'])

In [ ]:
nearby = get_systems_in_cube('Pho Aoscs OY-Z d13-10',100)
nearby.sort(key = lambda S:S['distance'])

In [ ]:
[dict(**S, stars=[dict(name=B['name'], subtype=B['subType']) for B in get_edsm_info(S['name'])['bodies'] if B['type'] == 'Star']) for S in nearby if S['distance']<60]

In [ ]:
nearby_with_info = [dict(**S, stars=[dict(name=B['name'], subtype=B['subType']) for B in get_edsm_info(S['name'])['bodies'] if B['type'] == 'Star']) for S in nearby]

In [ ]:
set([S.get('name') for S in nearby_with_info for B in S.get('stars') if B.get('subtype') == 'Neutron Star' ])

In [ ]:
{k:v for k,v in zip(['x','y','z'], current.get('coords'))}

In [ ]:
nearby.sort(key = lambda S:S['distance'])

In [ ]:
nearby_with_info[0]

In [ ]:
? nearby.sort

In [ ]:
target.get('name')

## edastro

In [ ]:
get_edsm_info('Pho Aoscs OY-Z d13-10')

In [ ]:
pd.DataFrame(route)

In [ ]:
prettyprint(get_edsm_info('Drojia TB-S c18-0'))

# Routing V1

In [ ]:
## interesting systems
targetsystems = dict(
    sol='Sol',
    guardian_fsd1='HD 63154',
    guardian_fsd2='Synuefe PX-J c25-8',
    mel_brandon='Luchtaine',
    marsha_hicks='Tir',
    cygnus_x1='V1357 Cygni',
    siteseeing_01='Phua Bre FB-O e6-257',
    paradox_destiny='Prai Hypoo TX-B d4', # carrier
    scorch_red_moon='Blue Hypooe VV-A c2-12',
    inverness='Thraikoo PS-U e2-4',       # carrier
    maerzenbecher='Hedgo GL-P c5-4',      # carrier
    sanctuary='Syreadiae JX-F c0',        # carrier
    sagitarius_a='Sagittarius A*',
    traikeou='Traikeou WL-I c11-1',
    thors_eye="Thor's Eye",
    annihilator='Great Annihilator',
    nearby_station='Prielo TZ-G d10-4',
    four_of_a_kind='Dryao Phylio AA-A h410'
)

In [ ]:
target_system = await find_system(targetsystems.get('nearby_station'))
end = np.round(np.asarray([target_system.get(k) for k in ["x","y","z"]]))

In [ ]:
target_system = await find_system(targetsystems.get('scorch_red_moon'))
end = np.round(np.asarray([target_system.get(k) for k in ["x","y","z"]]))

In [ ]:
target_system = await find_system(targetsystems.get('guardian_fsd1'))
end = np.round(np.asarray([target_system.get(k) for k in ["x","y","z"]]))

In [ ]:
cube_side = 60

In [ ]:
start_system = await find_system(system_name)
start = np.round(np.asarray([start_system.get(k) for k in ["x","y","z"]]))
path = [start]
system_names = [(await find_system(start.tolist())).get('name')]
path_info = {}


In [ ]:
pause_at = len(path)+5
travel_distance = np.sqrt(np.sum(np.square(end-path[-1])))
print(system_names[-1], path[-1])
print(f"\t{round(travel_distance)} ly -> {(await find_system(end.tolist())).get('name')} {np.round(end).tolist()}")

cube_side = round(2 * math.floor(cube_side/2))


async with pgpool.acquire() as pgconnection:
    
    while travel_distance > 2*cube_side and len(path) < pause_at:
        currentwp = np.round(path[-1],2)

        travel_distance = np.sqrt(np.sum(np.square(end - currentwp)))
        direction = (end - currentwp)/travel_distance
        nextwp = np.round(currentwp + 6*cube_side * direction,2)

        #print(f"Travelling from {system_names[-1]} {currentwp} in direction {np.round(direction,2)} towards {np.round(nextwp,2)}")

        cube_location = [round(cube_side*math.floor(v/cube_side)) for v in currentwp]
        cube_destination = [round(cube_side*math.floor(v/cube_side)) for v in nextwp]

        enclosure = [
            min(cube_location[0], cube_destination[0]) - 2*cube_side, max(cube_location[0], cube_destination[0]) + 2*cube_side + 1,
            min(cube_location[1], cube_destination[1]) - 4*cube_side, max(cube_location[1], cube_destination[1]) + 4*cube_side + 1,
            min(cube_location[2], cube_destination[2]) - 2*cube_side, max(cube_location[2], cube_destination[2]) + 2*cube_side + 1,
        ]    
        
        t = [int(x) for x in enclosure]
        extend = [x for x in zip(t[::2], t[1::2])]  

        #print(enclosure)
        #print(extend)
        
        full = np.unique(np.asarray([(round(cx + cube_side/2),round(cy + cube_side/2),round(cz + cube_side/2))  
                                         for cx in range(*extend[0], cube_side) 
                                         for cy in range(*extend[1], cube_side) 
                                         for cz in range(*extend[2], cube_side)], 
                                    dtype=[("cx","int64"),("cy","int64"),("cz","int64")]))
        
        #print('full', full.shape)
        #print(pd.DataFrame(full).head())

        candidate_cubes = await pgpool.fetch('''
            SELECT ROUND($7*FLOOR(x/$7) + $7/2) AS cx, ROUND($7*FLOOR(y/$7) + $7/2) AS cy, ROUND($7*FLOOR(z/$7) + $7/2) AS cz, 
                    count(1) starcount, 
                    ROUND(|/((AVG(x)-$8)^2 + (AVG(y)-$9)^2 + (AVG(z)-$10)^2)) distance,
                    0 weight
            FROM systems 
            WHERE x >= $1 AND x <= $2 AND  y >= $3 AND y <= $4  AND z  >= $5 AND z <= $6 
            GROUP BY cx, cy, cz
            ORDER BY starcount

        ''', *enclosure, cube_side, *end.tolist())
        #print(pd.DataFrame([R for R in candidate_cubes]).head())
        
        regions = np.unique(np.asarray(
            [tuple(R) for R in candidate_cubes], 
            dtype=[("cx","int64"), ("cy","int64"),("cz","int64"),("starcount","float64"), ("distance","float64"), ("weight","float64")]))
        #print('regions', regions.shape)
        #print(pd.DataFrame(regions).head())

        assert not regions.shape[0] > full.shape[0]
        joined = join_by(
            ('cx', 'cy','cz'), 
            full,regions, 
            jointype="outer", usemask=False,
            defaults = {"starcount":0, "distance":np.nan, 'weight':np.nan}
        )

        #print('joined', joined.shape)

        n = ~np.isfinite(joined["distance"])

        #joined["starcount"][n] = 1
        joined["distance"][n] = np.round(np.sqrt(
            np.square(joined[n]["cx"]-end[0]) + 
            np.square(joined[n]["cy"]-end[1]) + 
            np.square(joined[n]["cz"]-end[2])))
        
        #joined['weight'][n] = np.round(np.sqrt(1+joined['starcount'][n]) * joined["distance"][n])
        #joined['weight'][~n] = np.round(np.sqrt(joined['starcount'][~n]) * joined["distance"][~n])
        #joined['weight'] = np.round(np.sqrt(1+joined['starcount']) * joined["distance"])
        #joined['weight'] = np.round(np.sqrt(3+joined['starcount']) * joined["distance"])
        joined['weight'] = np.round(np.log(3+joined['starcount']) * joined["distance"])
        #joined.sort(order=["distance"])
        #print("full\n",pd.DataFrame(joined[n]).head(7))
        #print("regions\n", pd.DataFrame(joined[~n]).head(7))

        joined.sort(order=['weight', "distance","starcount"])        
        #print(pd.DataFrame(joined).head(30))
        path_info[system_names[-1]] = {'cubes':np.asarray(joined[0:3]), 'candidates':[], 'stations':[]}
        
        if candidate_cubes:
            candidates = []
            s = 30
            while not candidates and s < 300:
                s += 20
                candidates = await find_nearby_systems([joined[0][k] for k in ["cx","cy","cz"]], s)
                
            candidate = candidates[0]
            #print(candidate)
            path_info[system_names[-1]]['candidates'].append(record_to_dict(candidate))
            #path_info[system_names[-1]]['stations'] += [(R.get('system'),R.get('station'), round(R.get('distance'))) for R in await find_nearby_stations([joined[0][k] for k in ["cx","cy","cz"]],300)]
            if candidate.get("name") in system_names:
                break
                
            if np.round(np.sqrt(np.sum(np.square(path[-1]-end)))) < 2*cube_side+np.round(np.sqrt(np.sum(np.square(np.asarray([candidate.get(k) for k in ["x","y","z"]])-end)))):
                break

            path.append(np.asarray([candidate.get(k) for k in ["x","y","z"]]))
            print(f"""{system_names[-1]:26} {np.sqrt(np.sum(np.square(path[-1]-currentwp))):.1f} ly towards\t{candidate.get('name'):26} ({round(joined[0]['starcount'])})\t{np.sqrt(np.sum(np.square(path[-1]-end))):.1f} ly remaining""")
             
            path_info[system_names[-1]]['nextwp']=dict(
                system=candidate.get('name'),
                distance=np.round(np.sqrt(np.sum(np.square(path[-1]-currentwp))),1),
                remaining=np.round(np.sqrt(np.sum(np.square(path[-1]-end))),1),
                weight=round(np.sqrt(np.sum(np.square(path[-1]-end)))/joined[0]['weight'],2),
                density=joined[0]['starcount']/np.power(cube_side/3.26,3),
                cube=[joined[0][k] for k in ["cx","cy","cz"]],
                error=np.round(np.sqrt(np.sum(np.square(path[-1]-[joined[0][k] for k in ["cx","cy","cz"]]))))
            )
            system_names.append(candidate.get("name"))
        else:
            break

In [ ]:
pd.DataFrame([
    (
        W,I.get('distance'), I.get('system'), I.get('remaining'), 
        I.get('error'),I.get('weight'),I.get('density'), 
        I.get('cube',[np.nan,np.nan,np.nan])[1]) 
    for W,I in {WP:N.get('nextwp',{}) for WP, N in path_info.items()}.items()],
    columns=['waypoint', 'distance', 'nextwp', 'remaining','error','weight','density','y']
).set_index('waypoint')

In [ ]:
async with aiohttp.ClientSession() as session:

    info = await get_edsm_info(session, system_name)
    prettyprint(info)

## Navigation Route

In [ ]:
logpath = "/Users/fenke/Saved Games/Frontier Developments/Elite Dangerous"
logfile = os.path.join(logpath, "NavRoute.json")
print(logfile)
route = []
edsm_info={}

async with aiohttp.ClientSession() as session:

    count = 0

    with open(logfile, "rt") as jsonfile:
        navroute = json.load(jsonfile)
        #prettyprint(navroute)

        for item in navroute.get('Route'):
            N = item.get('StarSystem')
            #print(N, *[c for c in item.get('StarPos')])
            #assert False
            await pgpool.execute(
                """INSERT INTO eddb.systems (name, x, y, z) 
                    VALUES ($1, $2, $3, $4) 
                    ON CONFLICT DO NOTHING
                """,
                N, *[c for c in item.get('StarPos')]
            )

            if N not in edsm_info_cache:
                async with session.get('https://www.edsm.net/api-system-v1/bodies', params=dict(systemName=N)) as req:
                    edsm_info_cache[N] = await req.json()
            edsm_info = await get_edsm_info(session, N)
            route.append([
                '*' if system_name==N else ">" if not edsm_info else " " ,
                "#" if item.get('StarClass') in ['A','F','G'] else " ",
                f"{edsm_info.get('name'):26}" if edsm_info else f"{item.get('StarSystem'):26}",
                f"{systems.get(N,item).get('StarClass',([B.get('subType').replace('Star', '') for B in edsm_info.get('bodies',[{}]) if B.get('isMainStar')]+[''])[0])}", 
                f"{len([B.get('discovery') for B in edsm_info.get('bodies',[])])} / {edsm_info.get('bodyCount',len(systems.get(N,{}).get('bodies',{})))}",
                "|".join([
                    B.get('name','').replace(edsm_info.get('name'),'') 
                    for B in (edsm_info.get('bodies',[]) if edsm_info else systems.get(N,{}).get('bodies',{}).values())
                    for k in ['earth', 'water', 'ammonia', 'metal'] 
                    if k in B.get('subType','').lower()]),

            ])



pd.DataFrame(
    route, 
    columns=["d", "s", "system","class", "bodies", "valuable"])       

In [ ]:
systems.get('Zejai KY-Z d13-3')

In [ ]:
bodies.get(N)

In [ ]:
dict().

# Scratchpad


In [ ]:
neutron_planes = await pgpool.fetch('''
    SELECT ROUND($5*FLOOR(y/$5) + $5/2) AS cy, 
            count(*) starcount, 
            count(*) filter (WHERE ) as neutron_count
    FROM systems 
    WHERE x >= $1 AND x <= $2 AND z  >= $3 AND z <= $4
    GROUP BY cy

''', *enclosure, cube_side, *end.tolist())


In [ ]:
from scipy import stats
import matplotlib.pyplot as plt

def plot(x, y=None, fname=None, **kwargs):

    '''

    Create a 1 dimensional x-plot or 2-dimensional (y versus x) plot.

    Parameters
    x: The x coordinates of the points or line nodes.
    y: The y coordinates of the points or line nodes.

    '''

    # Only plot x-coördinates.
    if y is None:
        plt.plot(x, **kwargs)

    # Plot both x and y coördinates.
    else:
        plt.xticks(rotation=45)
        plt.plot(x, y, **kwargs)
    if fname is not None:
        plt.savefig(fname)


def duoplot(x1, y1, x2, y2, xlabel, y1label, y2label, **kwargs):

    fig, ax1 = plt.subplots(**kwargs)
    plt.xticks(rotation=45)

    color = 'tab:orange'
    ax1.set_xlabel(xlabel)
    ax1.set_ylabel(y1label, color=color)
    l1, *a1 = ax1.plot(x1, y1, color=color, label=y1label)
    ax1.tick_params(axis='y', labelcolor=color)

    ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

    color = 'tab:blue'
    ax2.set_ylabel(y2label, color=color)  # we already handled the x-label with ax1
    l2, *a2 = ax2.plot(x2, y2, color=color, label=y2label)
    ax2.tick_params(axis='y', labelcolor=color)

    plt.legend(handles=[l1,l2])
    fig.tight_layout()  # otherwise the right y-label is slightly clipped
    plt.show()

    return fig, ax1, ax2


In [ ]:
get_commander_position('immerlicht', os.getenv('EDSM_TOKEN'))

In [ ]:
#def star_distribution(side, x_between, z_between):
cube_side = 120
x_between = [-4000, 4000]
z_between = [-0, 4000]

y_extend_record = dict(await pgpool.fetchrow('''
    SELECT min(y), max(y)
    FROM eddb.systems
    where y < 3000 and y > -3000
'''))

cube_side = int(2 * np.round(cube_side/2))

y_between = [int(cube_side * np.ceil(y_extend_record['min']/cube_side)), int(cube_side * np.floor(y_extend_record['max']/cube_side))]
x_between = [int(cube_side * np.ceil(min(*x_between)/cube_side)), int(cube_side * np.floor(max(*x_between)/cube_side))]
z_between = [int(cube_side * np.ceil(min(*z_between)/cube_side)), int(cube_side * np.floor(max(*z_between)/cube_side))]

extend = [x_between, y_between, z_between]
    


In [ ]:
# distribution along Axis
# 
galactic_axis = 'y'                     
candidate_cubes = await pgpool.fetch(f'''
    SELECT ROUND($7*FLOOR({galactic_axis}/$7) + $7/2) AS c{galactic_axis}, 
            count(1) filter(where not n) as starcount, 
            count(*) filter(where n) as ncount
    FROM systems 
    WHERE x >= $1 AND x <= $2 AND  y >= $3 AND y <= $4  AND z  >= $5 AND z <= $6 
    GROUP BY c{galactic_axis}

''', *extend[0], *extend[1], *extend[2], cube_side)

regions = np.unique(np.asarray(
    [tuple(R) for R in candidate_cubes], 
    dtype=[(f"c{galactic_axis}","int64"),("starcount","float64"), ("ncount","float64")]))

stardistdata = np.zeros(shape=(regions.shape[0],len(regions.dtype)+1))
stardistdata[:,len(regions.dtype)] = np.log(1+ (1+regions['ncount'])/(1+regions['starcount']))
for ci, cn in zip(range(len(regions.dtype)), regions.dtype.names) :
    stardistdata[:,ci] = regions[cn]


In [ ]:
plot(stardistdata[:,0], np.log((1+stardistdata[:,1])/(1+stardistdata[:,2])))

In [ ]:
duoplot(stardistdata[:,0], stardistdata[:,1],stardistdata[:,0], stardistdata[:,2], f"{galactic_axis}", 'stars', 'neutron')

In [ ]:
duoplot(stardistdata[:,0], np.log(1+stardistdata[:,1]),stardistdata[:,0], np.log(1+stardistdata[:,2]), f"{galactic_axis}", 'stars', 'neutron')

In [ ]:
class L(object):
    ldata = "L"
    def get_rdata(self):
        return self.get_data()
    
class R(object):
    rdata = "R"
    def get_ldata(self):
        return self.get_data()
    
class C(L,R):
    cdata = "C"
    def get_data(self):
        return self.ldata + self.rdata

In [ ]:
c = C()

In [ ]:
c.get_ldata()

In [ ]:
def build_query(select_obj):
    buildquery_context = {}
    the_query = f"""
        SELECT {build_select(select_obj.get('select',[]))}
        FROM   {build_from_item(select_obj.get('from',[]))}
        WHERE  {build_condition(select_obj.get('where',[]))}
    """
    
    return the_query

In [ ]:
def build_select(select_items):
    if isinstance(select_items, str):
        return select_items
    #elif isinstance(select_items, list):
    
    return select_items

In [ ]:
    
def build_from_item(from_item):
    if not from_item:
        return ' '
    if isinstance(from_item, str):
        return f" {str(from_item)} "
    elif isinstance(from_item,list):
        return f" {str(from_item[0])} AS {str(from_item[1])} "
    elif isinstance(from_item, dict):
        return f""" 
            {build_from_item(from_item.get('items')[0])} 
            {str(from_item.get('type')).upper()} {build_from_item(from_item.get('items')[1])}
            ON {build_condition(from_item.get('on'))}
        """ + f"""
            AS {str(from_item.get('as'))}
        """ if str(from_item.get('as', '')) else ""
        

In [ ]:
def build_condition(condition):
    return condition

In [ ]:
build_from_item(
        "systems "+
        "LEFT JOIN populated " +
        "ON systems.name = populated.systemname "

)

In [ ]:
d = {1:2}

In [ ]:
d.pop(2, 0)

In [ ]:
print(build_query({
    "select":"*",
    "from": {
        "items":['systems', 'populated'],
        "type":'inner join',
        "on":'systems.name = populated.systemname'
    }
}))

In [ ]:
async with aiohttp.ClientSession() as session:
    
    edsm_info = await get_edsm_info(session, 'HIP 75281')

In [ ]:
edsm_info['bodies'][0]

In [ ]:
[(N) for N,S in edsm_info_cache.items() for B in S.get('bodies',[] if B.get('isMainStar'))

In [ ]:
logpath = "/Users/fenke/Saved Games/Frontier Developments/Elite Dangerous"


def edc_track_journal(journalpath=logpath, journalglob="journal.22*"):
    # Generator

    try:
        logfiles = sorted(glob.glob(os.path.join(logpath, "journal.22*.01.log")))
        current_journal = logfiles[-1]

        with open(current_journal, "rt") as journalfile:
            line = journalfile.readline()
            item = json.loads(line[0:-2]) if line[-2] == "," else json.loads(line)
            print(
                f"{'Odyssey' if item.get('Odyssey') else 'Horizons'}",
                f"version {item.get('gameversion')}"
                f"\nCurrent Journal: {current_journal}"
            ) 

            while True:# not shutdown_seen:
                line = journalfile.readline()
                if not line:
                    time.sleep(0.3)
                    continue

                if len(line) < 5:
                    continue

                item = json.loads(line[0:-2]) if line[-2] == "," else json.loads(line)

                yield item
                if item.get('event', '') == 'Shutdown':
                    print(f"Shutdown")
                    break

    except KeyboardInterrupt as kbi:
        print(f"Keyboard Interrupt")
        pass

                        
                        

In [ ]:
logger = ed_follow_journal()
for I in logger:
    pass
    #print(I.get('event'))

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
import inspect

In [ ]:
inspect.signature(find_system)

# Analyzing bodies

We want to get data that may tell us if there's a pattern relating
Starclass with materials found on planets - preferably with 
'signals'/'volcanism'

In [ ]:
planet_values = {
    'Water world': 415613,
    'Earth-like world': 1126206,
    'Ammonia world': 597762,
}

## First Attempt

In [ ]:
filename = "E:/data/eddb/galaxy_1day.json.gz"
#filename = "E:/data/eddb/galaxy_7days.json.gz"
#filename = "E:/data/eddb/galaxy_1month.json.gz"


spectral_class_list = set(sorted([
    'A0', 'A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 
    'AeBe0', 'AeBe1', 'AeBe2', 'AeBe3', 'AeBe4', 'AeBe5', 'AeBe6', 'AeBe7', 'AeBe8', 'AeBe9', 
    'B0', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 
    'C1', 'C5', 'CJ5', 'CN5', 
    'F', 'F0', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 
    'G0', 'G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9', 
    'K0', 'K1', 'K2', 'K3', 'K4', 'K5', 'K6', 'K7', 'K8', 'K9', 
    'L0', 'L1', 'L2', 'L3', 'L4', 'L5', 'L6', 'L7', 'L8', 'L9', 
    'M0', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 
    'MS0', 'MS2', 'MS3', 'MS4', 'MS5', 
    'O0', 'O5', 'O6', 'O7', 'O8', 'O9', 
    'S0', 'S1', 'S2', 'S3', 'S4', 'S5', 
    'T0', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 
    'TTS0', 'TTS1', 'TTS2', 'TTS3', 'TTS4', 'TTS5', 'TTS6', 'TTS7', 'TTS8', 'TTS9', 
    'W5', 'WC0', 'WC5', 'WN0', 'WNC0', 'WO0', 
    'Y0', 'Y1', 'Y2', 'Y3', 'Y4', 'Y5', 'Y6',
    'CJ4', 'CN4', 'K', 'MS1',
    'C0', 'C3', 'C4', 'C6', 'O4', 'W6', 'W9', 'WN5'
]))
spectral_class_map = {s:i for i,s in zip(range(len(spectral_class_list)), spectral_class_list)}

luminosity_list = set(sorted([
    'I', 'II', 'III', 'IIIa', 'IIIab', 'IIIb', 'IIa', 'IIab', 
    'IV', 'IVa', 'IVab', 'IVb', 'Ia0', 'Iab', 'Ib', 
    'O', 'V', 'VI', 'VII', 'Va', 'Vab', 'Vb', 'Vz',
    'IIb', 'Ia'
]))
luminosity_map = {s:i for i,s in zip(range(len(luminosity_list)), luminosity_list)}

magnitude_list = set([str(m) for m in range(-22,27)])
magnitude_map = {s:i for i,s in zip(range(len(magnitude_list)), magnitude_list)}

filesize=Path(filename).stat().st_size
chunksize = 64 * 1024 * 1024
est_count = int(7*filesize/chunksize) + 1
print(f"Reading {filename}, {round(filesize/(1024*1024),1)} Mb in approx {est_count} chunks")

#itemdb =[]
data = []

spectral_class_map = recreate_map(spectral_class_map)
luminosity_map = recreate_map(luminosity_map)
magnitude_map = recreate_map(magnitude_map)
terraform_map = {}

mapping = dict(
    spectralClass=lambda X: spectral_class_map.get(X,0),
    luminosity=lambda X: luminosity_map.get(X,0),
    absoluteMagnitude=lambda X: round(X)
)

dumped = dict(systems=0, count=0, total=0)
missed = dict(
    spectral=set(),
    luminosity=set(),
    magnitude=set(),
    terraform=set()
)
mapcat = dict(
    spectral=spectral_class_map,
    luminosity=luminosity_map,
    magnitude=magnitude_map,
    terraform=terraform_map
)
def catmap(cat, value):
    if value not in mapcat[cat]:
        missed[cat].add(value)
        return np.nan
    else:
        return mapcat[cat][value]
    
#totals = {C:dict(systems=0, count=0, total=0) for C in spectral_class_map }
totals = {C:dict(systems=0, count=0, total=0) for C in set([s[:-1] for s in spectral_class_map]) }

count = 0
system_count = 0
columns = slice(2,6)
start = time.process_time()
with gzip.open(filename, 'rt') as jsonfile:

    firstline = jsonfile.readline()

    while True:
        
        count += 1
        chunk = jsonfile.readlines(chunksize)
        if chunk:
            for line in chunk:
                if len(line) < 5:
                    continue
                # This is where we decode the item and get data from it ----------------
                item = json.loads(line[0:-2]) if line[-2] == "," else json.loads(line)
                system_count += 1
                #itemdb.append(item)
                #continue
                # get the data we need
                bodies = item.get('bodies',[])
                if not bodies:
                    continue
                if item.get('bodyCount',0) != len(bodies):
                    continue
                    
                mainstars = [B for B in bodies if B.get('mainStar') and B.get('spectralClass')]
                if not mainstars:
                    continue
                mainstar = mainstars[0]
                data.append(
                    [ # Xn
                        item.get('name',''),
                        item.get('bodyCount',len(bodies)),
                        len([B for B in bodies if B.get('spectralClass') and not B.get('mainStar') ]),
                        round(sum([B.get('solarMasses') for B in bodies if B.get('spectralClass')])),
                        #spectral_class_map.get(mainstar.get('spectralClass'),np.nan),
                        #luminosity_map.get(mainstar.get('luminosity'),np.nan),
                        #magnitude_map.get(str(round(mainstar.get('absoluteMagnitude'))),np.nan),
                        catmap('spectral', mainstar.get('spectralClass')),
                        catmap('luminosity', mainstar.get('luminosity')),
                        catmap('magnitude', str(round(mainstar.get('absoluteMagnitude')))),
                        int(bool([True for b in bodies if b.get('terraformingState') == 'Terraformable'])), # y1
                        int(bool([True for b in bodies if b.get('subType') in planet_values])) # y2
                    ])
                
                    
            # keep our programmer up to date of progress
            print(f"{count}/{est_count}\t{100*count/est_count:3.2f}%, {int(system_count / (time.process_time() - start)):6} /s,\t{system_count:9} systems, {((est_count - count) * (time.process_time() - start)/count):5.1f} seconds remaining")
            continue
            
        print(f"Empy chunk -> Done! Imported {system_count} systems in {round(time.process_time() - start,1)} seconds")
        break

            
adata = np.asarray(np.asarray(data)[:,1:], dtype=float)
tpl = (time.process_time() - start)/system_count
print(f"{ (time.process_time() - start)} seconds {system_count} systems, per system {round(1000000*tpl,2)} us, est: {tpl*51854708}") 

In [ ]:
missed

## Next Attempt

In [ ]:
spectral_class_list = set(sorted([
    'A0', 'A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 
    'AeBe0', 'AeBe1', 'AeBe2', 'AeBe3', 'AeBe4', 'AeBe5', 'AeBe6', 'AeBe7', 'AeBe8', 'AeBe9', 
    'B0', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 
    'C1', 'C5', 'CJ5', 'CN5', 
    'F', 'F0', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 
    'G0', 'G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9', 
    'K0', 'K1', 'K2', 'K3', 'K4', 'K5', 'K6', 'K7', 'K8', 'K9', 
    'L0', 'L1', 'L2', 'L3', 'L4', 'L5', 'L6', 'L7', 'L8', 'L9', 
    'M0', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 
    'MS0', 'MS2', 'MS3', 'MS4', 'MS5', 
    'O0', 'O5', 'O6', 'O7', 'O8', 'O9', 
    'S0', 'S1', 'S2', 'S3', 'S4', 'S5', 
    'T0', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 
    'TTS0', 'TTS1', 'TTS2', 'TTS3', 'TTS4', 'TTS5', 'TTS6', 'TTS7', 'TTS8', 'TTS9', 
    'W5', 'WC0', 'WC5', 'WN0', 'WNC0', 'WO0', 
    'Y0', 'Y1', 'Y2', 'Y3', 'Y4', 'Y5', 'Y6',
    'CJ4', 'CN4', 'K', 'MS1',
    'C0', 'C3', 'C4', 'C6', 'O4', 'W6', 'W9', 'WN5'
]))
spectral_class_map = {s:i for i,s in zip(range(len(spectral_class_list)), spectral_class_list)}

In [ ]:
luminosity_list = set(sorted([
    'I', 'II', 'III', 'IIIa', 'IIIab', 'IIIb', 'IIa', 'IIab', 
    'IV', 'IVa', 'IVab', 'IVb', 'Ia0', 'Iab', 'Ib', 
    'O', 'V', 'VI', 'VII', 'Va', 'Vab', 'Vb', 'Vz',
    'IIb', 'Ia'
]))
luminosity_map = {s:i for i,s in zip(range(len(luminosity_list)), luminosity_list)}

In [ ]:
magnitude_list = sorted([m for m in range(-15,27)])
magnitude_map = {s:i for i,s in zip(range(len(magnitude_list)), magnitude_list)}

In [ ]:
main_sequence=["O", "B", "A", "F", "G", "K", "M"]
main_subclass=[str(c) for c in range(10)]
main_sequence_map = {s:i for i,s in zip(range(len(main_sequence)), main_sequence)}
main_subclass_map = {s:i for i,s in zip(range(len(main_subclass)), main_subclass)}

main_spectral_map={
    q+c:dict(seq=main_sequence_map[q], sub=main_subclass_map[c]) 
    for q in main_sequence for c in main_subclass
    if q+c in spectral_class_map
}
#  {s:i for i,s in zip(range(len(templist)), templist)}

In [ ]:
main_luminosity=[l for l in [
    l for l in [
        m+s for m in ['I','II','III','IV','V','VI','VII','VIII'] for s in ['z','a', 'ab','','b']
    ] if l in luminosity_map] ]
main_luminosity_map = {s:i for i,s in zip(range(len(main_luminosity)), main_luminosity) }

#main_luminosity_map.update({k+'b':v for k,v in main_luminosity_map.items() if 'a' in k})
    

In [ ]:
main_luminosity_map

In [ ]:
planet_values = {
    'Water world': 415613,
    'Earth-like world': 1126206,
    'Ammonia world': 597762,
}

In [ ]:
#filename = "E:/data/eddb/galaxy_1day.json.gz"
#filename = "E:/data/eddb/galaxy_7days.json.gz"
#filename = "E:/data/eddb/galaxy_1month.json.gz"
filename = "E:/data/eddb/galaxy.json.gz"

filesize=Path(filename).stat().st_size
chunksize = 64 * 1024 * 1024
est_count = int(6.5*filesize/chunksize) + 1
print(f"Reading {filename}, {round(filesize/(1024*1024),1)} Mb in approx {est_count} chunks")

#itemdb =[]
data = []

missed = dict(
    spectral=set(),
    subclass=set(),
    luminosity=set(),
    magnitude=set(),
    terraform=set()
)
mapcat = dict(
    spectral=main_spectral_map,
    luminosity=main_luminosity_map,
    magnitude=magnitude_map)

def catmap(cat, value):
    if value not in mapcat[cat]:
        missed[cat].add(value)
        return None
    else:
        return mapcat[cat][value]
    
#totals = {C:dict(systems=0, count=0, total=0) for C in spectral_class_map }
totals = {C:dict(systems=0, count=0, total=0) for C in set([s[:-1] for s in spectral_class_map]) }

count = 0
system_count = 0
columns = slice(2,6)
start = time.process_time()
with gzip.open(filename, 'rt') as jsonfile:

    firstline = jsonfile.readline()

    while True:
        
        count += 1
        chunk = jsonfile.readlines(chunksize)
        if chunk:
            for line in chunk:
                if len(line) < 5:
                    continue
                # This is where we decode the item and get data from it ----------------
                item = json.loads(line[0:-2]) if line[-2] == "," else json.loads(line)
                system_count += 1
                #itemdb.append(item)
                #continue
                # get the data we need
                bodies = item.get('bodies',[])
                if not bodies:
                    continue
                if item.get('bodyCount',0) != len(bodies):
                    continue

                mainstars = [B for B in bodies if B.get('mainStar') and B.get('spectralClass')]
                if not mainstars:
                    continue
                mainstar = mainstars[0]
                
                if mainstar.get('spectralClass') not in  main_spectral_map:
                    continue
                if mainstar.get('luminosity') not in main_luminosity_map:
                    continue
                if mainstar.get('absoluteMagnitude') < -15:
                    continue
                    
                data.append(
                    [ # Xn
                        #item.get('name',''), 
                        item.get('bodyCount'),
                        len([B for B in bodies if B.get('spectralClass')]),
                        #int(round(2*sum([B.get('solarMasses') for B in bodies if B.get('spectralClass')]))),
                        int(round(np.sqrt(sum([B.get('solarMasses') for B in bodies if B.get('spectralClass')])))),
                        main_spectral_map.get(mainstar.get('spectralClass'))['seq'],
                        main_spectral_map.get(mainstar.get('spectralClass'))['sub'],
                        main_luminosity_map.get(mainstar.get('luminosity')),
                        magnitude_map.get(int(round(mainstar.get('absoluteMagnitude')))),
                        #int(round(mainstar.get('absoluteMagnitude'))),
                        #int(bool([True for b in bodies if b.get('terraformingState') == 'Terraformable'])), # y1
                        int(bool(sum([1 for b in bodies if b.get('terraformingState') == 'Terraformable'])>.5)), # y1
                        int(bool(sum([planet_values.get(b.get('subType'),0) for b in bodies])>400000)) # y2
                    ])
                
                    
            # keep our programmer up to date of progress

            sys.stdout.write(f"{count}/{est_count}\t{100*count/est_count:3.2f}%, {int(system_count / (time.process_time() - start)):6} /s, {system_count:9} systems, {((est_count - count) * (time.process_time() - start)/count):5.1f} seconds remaining\r")
            continue
            
        print(f"\nEmpty chunk -> Done! Imported {system_count} systems in {round(time.process_time() - start,1)} seconds")
        break

 

           
adata = np.asarray(data, dtype=int)
adata[:,0] = adata[:,0]-1
adata[:,1] = adata[:,1]-1
#n = np.isfinite(adata[:,6])
#np.savetxt('data/mssystems.csv', adata[n], delimiter=',', fmt='%.2f')
tpl = (time.process_time() - start)/system_count
print(f"{ (time.process_time() - start)} seconds {system_count} systems, per system {round(1000000*tpl,2)} us, est: {tpl*51854708}") 

In [ ]:
np.min(adata, axis=0)

In [ ]:
n = np.isfinite(adata[:,6])
np.savetxt('data/systems_full.csv', adata, delimiter=',', fmt='%d')


In [ ]:
df = pd.DataFrame(adata)
# name, bodycount, starcount, starmass, spectralclass, luminosity, magnitude, terra, high_value

In [ ]:
df.info()

In [ ]:
n = np.greater(adata[:,7],0) | np.greater(adata[:,8],0)

In [ ]:
n = np.isfinite(adata[:,6])

In [ ]:
n=np.less(adata[:,2],3)

In [ ]:
np.amin(adata[:,2])

In [ ]:
pd.DataFrame(adata[n]).info()

In [ ]:
df.sample(12)

In [ ]:
async with aiohttp.ClientSession() as session:
    prettyprint(await get_edsm_info(session, 'Lyncis Sector KR-W b1-3'))

In [ ]:
df[df[5]>0].info()

In [ ]:
df[df[8]>0].sample(20)

In [ ]:
df[df[7]>0].sample(20)

## Write as csv

In [ ]:
df.info()

In [ ]:
df.sample(12)

In [ ]:
dfd = adata[:, 1:].astype(float)

In [ ]:
adata[:, 1:]

In [ ]:
dfd[np.isfinite(dfd)]

In [ ]:
df = pd.DataFrame(dfd[np.isfinite(dfd)])
# name, bodycount, starcount, starmass, spectralclass, luminosity, magnitude, terra, high_value

In [ ]:
df.info()

In [ ]:
df[[1,2,3,4,5,6,7,8]].to_csv('data/system_scan.csv', header=False, index=False)

## Read from csv

In [ ]:
adata = np.loadtxt('data/systems_full.csv', delimiter=',',dtype=int)

In [ ]:
adata

In [ ]:
pd.DataFrame(adata).info()

In [ ]:
pd.DataFrame(adata).sample(12)

## Prep & cleanup

In [ ]:
np.min(adata, axis=0)

In [ ]:
adata[:,0] = adata[:,0]-1

## TPOT Training

In [ ]:
from tpot import TPOTClassifier, TPOTRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, confusion_matrix, ConfusionMatrixDisplay

In [ ]:
import importlib
importlib.reload(tpot)
from tpot import TPOTClassifier, TPOTRegressor

In [ ]:
print(f"Checking CUDA: {torch.cuda.get_device_name(torch.cuda.current_device()) if torch.cuda.is_available() else 'No-Cuda'} ")

In [ ]:

classifier_config_cuml = {
    # cuML + DMLC/XGBoost Classifiers

    "cuml.neighbors.KNeighborsClassifier": {
        "n_neighbors": range(1, 101),
        "weights": ["uniform",],
    },

    "cuml.linear_model.LogisticRegression": {
        "penalty": ["l1", "l2", "elasticnet"],
        "C": [1e-4, 1e-3, 1e-2, 1e-1, 0.5, 1., 5., 10., 15., 20., 25.,],
    },

    # Sklearn Preprocesssors

    "sklearn.preprocessing.Binarizer": {
        "threshold": np.arange(0.0, 1.01, 0.05)
    },

    "sklearn.decomposition.FastICA": {
        "tol": np.arange(0.0, 1.01, 0.05)
    },

    "sklearn.cluster.FeatureAgglomeration": {
        "linkage": ["ward", "complete", "average"],
        "affinity": ["euclidean", "l1", "l2", "manhattan", "cosine"]
    },

    "sklearn.preprocessing.MaxAbsScaler": {
    },

    "sklearn.preprocessing.MinMaxScaler": {
    },

    "sklearn.preprocessing.Normalizer": {
        "norm": ["l1", "l2", "max"]
    },

    "sklearn.kernel_approximation.Nystroem": {
        "kernel": ["rbf", "cosine", "chi2", "laplacian", "polynomial", "poly", "linear", "additive_chi2", "sigmoid"],
        "gamma": np.arange(0.0, 1.01, 0.05),
        "n_components": range(1, 11)
    },

    "sklearn.decomposition.PCA": {
        "svd_solver": ["randomized"],
        "iterated_power": range(1, 11)
    },

    "sklearn.kernel_approximation.RBFSampler": {
        "gamma": np.arange(0.0, 1.01, 0.05)
    },

    "sklearn.preprocessing.RobustScaler": {
    },

    "sklearn.preprocessing.StandardScaler": {
    },

    "tpot.builtins.ZeroCount": {
    },

    "tpot.builtins.OneHotEncoder": {
        "minimum_fraction": [0.05, 0.1, 0.15, 0.2, 0.25],
        "sparse": [False],
        "threshold": [10]
    },

    # Selectors

    "sklearn.feature_selection.SelectFwe": {
        "alpha": np.arange(0, 0.05, 0.001),
        "score_func": {
            "sklearn.feature_selection.f_classif": None
        }
    },

    "sklearn.feature_selection.SelectPercentile": {
        "percentile": range(1, 100),
        "score_func": {
            "sklearn.feature_selection.f_classif": None
        }
    },

    "sklearn.feature_selection.VarianceThreshold": {
        "threshold": [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.2]
    }
}

classifier_config_nn = {

    'tpot.builtins.PytorchLRClassifier': {
        'learning_rate': [1e-3, 1e-2, 1e-1, 0.5, 1.],
        'batch_size': [4, 8, 16, 32],
        'num_epochs': [5, 10, 15],
        'weight_decay': [0, 1e-4, 1e-3, 1e-2]
    },

    'tpot.builtins.PytorchMLPClassifier': {
        'learning_rate': [1e-3, 1e-2, 1e-1, 0.5, 1.],
        'batch_size': [4, 8, 16, 32],
        'num_epochs': [5, 10, 15],
        'weight_decay': [0, 1e-4, 1e-3, 1e-2]
    },

    # Classifiers
    'sklearn.naive_bayes.GaussianNB': {
    },

    'sklearn.naive_bayes.BernoulliNB': {
        'alpha': [1e-3, 1e-2, 1e-1, 1., 10., 100.],
        'fit_prior': [True, False]
    },

    'sklearn.naive_bayes.MultinomialNB': {
        'alpha': [1e-3, 1e-2, 1e-1, 1., 10., 100.],
        'fit_prior': [True, False]
    },

    'sklearn.tree.DecisionTreeClassifier': {
        'criterion': ["gini", "entropy"],
        'max_depth': range(1, 11),
        'min_samples_split': range(2, 21),
        'min_samples_leaf': range(1, 21)
    },

    'sklearn.ensemble.ExtraTreesClassifier': {
        'n_estimators': [100],
        'criterion': ["gini", "entropy"],
        'max_features': np.arange(0.05, 1.01, 0.05),
        'min_samples_split': range(2, 21),
        'min_samples_leaf': range(1, 21),
        'bootstrap': [True, False]
    },

    'sklearn.ensemble.RandomForestClassifier': {
        'n_estimators': [100],
        'criterion': ["gini", "entropy"],
        'max_features': np.arange(0.05, 1.01, 0.05),
        'min_samples_split': range(2, 21),
        'min_samples_leaf':  range(1, 21),
        'bootstrap': [True, False]
    },

    'sklearn.ensemble.GradientBoostingClassifier': {
        'n_estimators': [100],
        'learning_rate': [1e-3, 1e-2, 1e-1, 0.5, 1.],
        'max_depth': range(1, 11),
        'min_samples_split': range(2, 21),
        'min_samples_leaf': range(1, 21),
        'subsample': np.arange(0.05, 1.01, 0.05),
        'max_features': np.arange(0.05, 1.01, 0.05)
    },

    'sklearn.neighbors.KNeighborsClassifier': {
        'n_neighbors': range(1, 101),
        'weights': ["uniform", "distance"],
        'p': [1, 2]
    },

    'sklearn.svm.LinearSVC': {
        'penalty': ["l1", "l2"],
        'loss': ["hinge", "squared_hinge"],
        'dual': [True, False],
        'tol': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1],
        'C': [1e-4, 1e-3, 1e-2, 1e-1, 0.5, 1., 5., 10., 15., 20., 25.]
    },

    'sklearn.linear_model.LogisticRegression': {
        'penalty': ["l1", "l2"],
        'C': [1e-4, 1e-3, 1e-2, 1e-1, 0.5, 1., 5., 10., 15., 20., 25.],
        'dual': [True, False]
    },

    'xgboost.XGBClassifier': {
        'n_estimators': [100],
        'max_depth': range(1, 11),
        'learning_rate': [1e-3, 1e-2, 1e-1, 0.5, 1.],
        'subsample': np.arange(0.05, 1.01, 0.05),
        'min_child_weight': range(1, 21),
        'n_jobs': [1],
        'verbosity': [0]
    },

    'sklearn.linear_model.SGDClassifier': {
        'loss': ['log', 'hinge', 'modified_huber', 'squared_hinge', 'perceptron'],
        'penalty': ['elasticnet'],
        'alpha': [0.0, 0.01, 0.001],
        'learning_rate': ['invscaling', 'constant'],
        'fit_intercept': [True, False],
        'l1_ratio': [0.25, 0.0, 1.0, 0.75, 0.5],
        'eta0': [0.1, 1.0, 0.01],
        'power_t': [0.5, 0.0, 1.0, 0.1, 100.0, 10.0, 50.0]
    },

    'sklearn.neural_network.MLPClassifier': {
        'alpha': [1e-4, 1e-3, 1e-2, 1e-1],
        'learning_rate_init': [1e-3, 1e-2, 1e-1, 0.5, 1.]
    },

    # Preprocesssors
    'sklearn.preprocessing.Binarizer': {
        'threshold': np.arange(0.0, 1.01, 0.05)
    },

    'sklearn.decomposition.FastICA': {
        'tol': np.arange(0.0, 1.01, 0.05)
    },

    'sklearn.cluster.FeatureAgglomeration': {
        'linkage': ['ward', 'complete', 'average'],
        'affinity': ['euclidean', 'l1', 'l2', 'manhattan', 'cosine']
    },

    'sklearn.preprocessing.MaxAbsScaler': {
    },

    'sklearn.preprocessing.MinMaxScaler': {
    },

    'sklearn.preprocessing.Normalizer': {
        'norm': ['l1', 'l2', 'max']
    },

    'sklearn.kernel_approximation.Nystroem': {
        'kernel': ['rbf', 'cosine', 'chi2', 'laplacian', 'polynomial', 'poly', 'linear', 'additive_chi2', 'sigmoid'],
        'gamma': np.arange(0.0, 1.01, 0.05),
        'n_components': range(1, 11)
    },

    'sklearn.decomposition.PCA': {
        'svd_solver': ['randomized'],
        'iterated_power': range(1, 11)
    },

    'sklearn.preprocessing.PolynomialFeatures': {
        'degree': [2],
        'include_bias': [False],
        'interaction_only': [False]
    },

    'sklearn.kernel_approximation.RBFSampler': {
        'gamma': np.arange(0.0, 1.01, 0.05)
    },

    'sklearn.preprocessing.RobustScaler': {
    },

    'sklearn.preprocessing.StandardScaler': {
    },

    'tpot.builtins.ZeroCount': {
    },

    'tpot.builtins.OneHotEncoder': {
        'minimum_fraction': [0.05, 0.1, 0.15, 0.2, 0.25],
        'sparse': [False],
        'threshold': [10]
    },

    # Selectors
    'sklearn.feature_selection.SelectFwe': {
        'alpha': np.arange(0, 0.05, 0.001),
        'score_func': {
            'sklearn.feature_selection.f_classif': None
        }
    },

    'sklearn.feature_selection.SelectPercentile': {
        'percentile': range(1, 100),
        'score_func': {
            'sklearn.feature_selection.f_classif': None
        }
    },

    'sklearn.feature_selection.VarianceThreshold': {
        'threshold': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.2]
    },

    'sklearn.feature_selection.RFE': {
        'step': np.arange(0.05, 1.01, 0.05),
        'estimator': {
            'sklearn.ensemble.ExtraTreesClassifier': {
                'n_estimators': [100],
                'criterion': ['gini', 'entropy'],
                'max_features': np.arange(0.05, 1.01, 0.05)
            }
        }
    },

    'sklearn.feature_selection.SelectFromModel': {
        'threshold': np.arange(0, 1.01, 0.05),
        'estimator': {
            'sklearn.ensemble.ExtraTreesClassifier': {
                'n_estimators': [100],
                'criterion': ['gini', 'entropy'],
                'max_features': np.arange(0.05, 1.01, 0.05)
            }
        }
    }
}

classifier_config_pytorch = {

    'tpot.builtins.PytorchLRClassifier': {
        'learning_rate': [1e-3, 1e-2, 1e-1, 0.5, 1.],
        'batch_size': [4, 8, 16, 32],
        'num_epochs': [5, 10, 15],
        'weight_decay': [0, 1e-4, 1e-3, 1e-2]
    },

    'tpot.builtins.PytorchMLPClassifier': {
        'learning_rate': [1e-3, 1e-2, 1e-1, 0.5, 1.],
        'batch_size': [4, 8, 16, 32],
        'num_epochs': [5, 10, 15],
        'weight_decay': [0, 1e-4, 1e-3, 1e-2]
    },


    # Preprocesssors
    'sklearn.preprocessing.Binarizer': {
        'threshold': np.arange(0.0, 1.01, 0.05)
    },

    'sklearn.decomposition.FastICA': {
        'tol': np.arange(0.0, 1.01, 0.05)
    },

    'sklearn.cluster.FeatureAgglomeration': {
        'linkage': ['ward', 'complete', 'average'],
        'affinity': ['euclidean', 'l1', 'l2', 'manhattan', 'cosine']
    },

    'sklearn.preprocessing.MaxAbsScaler': {
    },

    'sklearn.preprocessing.MinMaxScaler': {
    },

    'sklearn.preprocessing.Normalizer': {
        'norm': ['l1', 'l2', 'max']
    },

    'sklearn.kernel_approximation.Nystroem': {
        'kernel': ['rbf', 'cosine', 'chi2', 'laplacian', 'polynomial', 'poly', 'linear', 'additive_chi2', 'sigmoid'],
        'gamma': np.arange(0.0, 1.01, 0.05),
        'n_components': range(1, 11)
    },

    'sklearn.decomposition.PCA': {
        'svd_solver': ['randomized'],
        'iterated_power': range(1, 11)
    },

    'sklearn.preprocessing.PolynomialFeatures': {
        'degree': [2],
        'include_bias': [False],
        'interaction_only': [False]
    },

    'sklearn.kernel_approximation.RBFSampler': {
        'gamma': np.arange(0.0, 1.01, 0.05)
    },

    'sklearn.preprocessing.RobustScaler': {
    },

    'sklearn.preprocessing.StandardScaler': {
    },

    'tpot.builtins.ZeroCount': {
    },

    'tpot.builtins.OneHotEncoder': {
        'minimum_fraction': [0.05, 0.1, 0.15, 0.2, 0.25],
        'sparse': [False],
        'threshold': [10]
    },

    # Selectors
    'sklearn.feature_selection.SelectFwe': {
        'alpha': np.arange(0, 0.05, 0.001),
        'score_func': {
            'sklearn.feature_selection.f_classif': None
        }
    },

    'sklearn.feature_selection.SelectPercentile': {
        'percentile': range(1, 100),
        'score_func': {
            'sklearn.feature_selection.f_classif': None
        }
    },

    'sklearn.feature_selection.VarianceThreshold': {
        'threshold': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.2]
    },

    'sklearn.feature_selection.RFE': {
        'step': np.arange(0.05, 1.01, 0.05),
        'estimator': {
            'sklearn.ensemble.ExtraTreesClassifier': {
                'n_estimators': [100],
                'criterion': ['gini', 'entropy'],
                'max_features': np.arange(0.05, 1.01, 0.05)
            }
        }
    },

    'sklearn.feature_selection.SelectFromModel': {
        'threshold': np.arange(0, 1.01, 0.05),
        'estimator': {
            'sklearn.ensemble.ExtraTreesClassifier': {
                'n_estimators': [100],
                'criterion': ['gini', 'entropy'],
                'max_features': np.arange(0.05, 1.01, 0.05)
            }
        }
    }
}

In [ ]:
classifier_config_nn.pop('xgboost.XGBClassifier')


In [ ]:
adata.shape

In [ ]:
tpot_X.shape

In [ ]:
tpot_X, final_X, tpot_y, final_y = train_test_split(
    adata[:, [0,1,2,3,4,5,6]], adata[:,[7,8]],
    train_size=0.3)

In [ ]:
pipeline_optimizer_terra = TPOTClassifier(
    #config_dict=classifier_config_nn,
    config_dict='TPOT NN',
    generations=3,
    population_size=59,
    warm_start=True,
    n_jobs=-3,
    max_eval_time_mins=5,
    max_time_mins=4*60,
    verbosity=3
)


In [ ]:
pipeline_optimizer_terra.fit(tpot_X, tpot_y[:,0]) # Terraformable

pipeline_optimizer_terra.export('data/tpot_classifier_terra_1.py')

In [ ]:
pipeline_optimizer_terra.fit(tpot_X, tpot_y[:,0]) # Terraformable

pipeline_optimizer_terra.export('data/tpot_classifier_terra_2.py')

In [ ]:
exported_pipeline_terra = pipeline_optimizer_terra.fitted_pipeline_.steps[-1][1]

In [ ]:
pipeline_optimizer_value = TPOTClassifier(
    #config_dict=classifier_config_nn,
    config_dict='TPOT NN',
    generations=3,
    population_size=59,
    warm_start=True,
    n_jobs=-3,
    max_eval_time_mins=5,
    max_time_mins=4*60,
    verbosity=3
)


In [ ]:
pipeline_optimizer_value.fit(tpot_X, tpot_y[:,1]) # Value planets

pipeline_optimizer_value.export('data/tpot_classifier_value_1.py')

In [ ]:
pipeline_optimizer_value.fit(tpot_X, tpot_y[:,1]) # Value planets

pipeline_optimizer_value.export('data/tpot_classifier_value_2.py')

In [ ]:
exported_pipeline_value = pipeline_optimizer_value.fitted_pipeline_.steps[-1][1]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    adata[:, [0,1,2,3,4,5,6]], adata[:,7],
    train_size=0.7)

In [ ]:
pipeline_optimizer.fit(X_train, y_train)

In [ ]:
pipeline_optimizer.export('data/tpot_classifier_terra_1m.py')

In [ ]:
exctracted_best_model = pipeline_optimizer.fitted_pipeline_.steps[-1][1]
exctracted_best_model.fit(X_train, y_train)

In [ ]:
y_test_pred  = exctracted_best_model.predict(X_test)

In [ ]:
mean_squared_error(y_test,y_test_pred )

In [ ]:
confusion_matrix(y_test,y_test_pred)

## Testing the pipeline

In [ ]:
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import StandardScaler
from tpot.builtins import StackingEstimator


In [ ]:
training_features, testing_features, training_target, testing_target = train_test_split(
    final_X, final_y,
    train_size=0.8)


In [ ]:
training_features, testing_features, training_target, testing_target = train_test_split(
    adata[:,[0,1,2,3,4,5,6]], adata[:,[7,8]],
    train_size=0.7)


### Value planets

In [ ]:
# Average CV score on the training set was: 0.9119772778209425
from sklearn.ensemble import ExtraTreesClassifier

exported_pipeline_values = ExtraTreesClassifier(
    bootstrap=True, criterion="gini", max_features=0.8500000000000001, 
    min_samples_leaf=18, min_samples_split=8, n_estimators=100)

In [ ]:
exported_pipeline_values.fit(training_features, training_target[:,1])
predicted_target_values = exported_pipeline_values.predict(testing_features)

In [ ]:
mean_squared_error(testing_target[:,1], predicted_target_values)

In [ ]:
confusion_matrix(testing_target[:,1], predicted_target_values)

In [ ]:
ConfusionMatrixDisplay.from_predictions(testing_target[:,1], exported_pipeline_values.predict(testing_features))

In [ ]:
tn, fp, fn, tp = confusion_matrix(testing_target[:,1], exported_pipeline_values.predict(testing_features)).ravel()

print(f"Actually False: \t True Negatives: {tn:6}, False Positives: {fp:6}")
print(f"Actually True:  \t False Negatives:{fn:6}, True Positives:  {tp:6}")
print(f"Predicted       \t     False        \t   True    ")

### Terraforming

In [ ]:
# Average CV score on the training set was: 0.885480051857461
from sklearn.ensemble import GradientBoostingClassifier

exported_pipeline_terra = GradientBoostingClassifier(
    learning_rate=0.1, max_depth=8, max_features=1.0, 
    min_samples_leaf=15, min_samples_split=7, 
    n_estimators=100, subsample=0.4)


In [ ]:
exported_pipeline_terra.fit(training_features, training_target[:,0])
predicted_target_terra = exported_pipeline_terra.predict(testing_features)

In [ ]:
mean_squared_error(testing_target[:,0], predicted_target_terra)

In [ ]:
ConfusionMatrixDisplay.from_predictions(testing_target[:,0], predicted_target_terra)

In [ ]:
tn, fp, fn, tp = confusion_matrix(testing_target[:,0], exported_pipeline_terra.predict(testing_features)).ravel()

print(f"Actually False: \t True Negatives: {tn:6}, False Positives: {fp:6}")
print(f"Actually True:  \t False Negatives:{fn:6}, True Positives:  {tp:6}")
print(f"Predicted       \t     False        \t   True    ")

In [ ]:
testing_features.shape, testing_target.shape

In [ ]:
testing_features[np.greater(testing_target[:,0],0.5)].shape

In [ ]:
results[np.greater(results,0)]

In [ ]:
y_test[y_test[7]>0]

In [ ]:
df[df[4]>0]

In [ ]:
df[df[4]>0]

In [ ]:
planet_values

In [ ]:
l = {}

In [ ]:
0 if l.update(dict(a=3)) else l.get('a')

In [ ]:
l

In [ ]:
min_count = system_count / 1000
pd.DataFrame([
    (
        k,
        round(100*v['count']/v['systems']),
        v['count'],
        v['systems'],
        v['total'],
        round(v['total'] / v['count'])
        
    )
    for k,v in totals.items()
    if v.get('count')>0 and v['count']/v['systems'] > .1 and v['systems'] > min_count
], columns=['spectralclass', 'chance', 'count', 'systems', 'total', 'average']).set_index(
    ['spectralclass']).sort_values(['chance'])

In [ ]:
    chance	count	total	average
class				
MS	11	33	20731734	628234
CN	12	27	16021355	593384
G	13	10232	6778211922	662452
AeBe	14	191	134299067	703136
F	17	19025	12397439067	651639
A	18	10030	6386976681	636787

In [ ]:
min_count

In [ ]:
prettyprint({k:v for k,v in totals.items() if v.get('count')>0})

In [ ]:
dumped

## Create mappings

In [ ]:
['spectralClass','luminosity','absoluteMagnitude']

In [ ]:
def auto_map(set_to_map):
    templist = sorted(set_to_map)
    return {s:i for i,s in zip(range(len(templist)), templist)}

def recreate_map(mapping):
    subval = min(mapping.values())
    if subval > 0:
        mapping = {k:v-subval for k,v in mapping.items()}

    return mapping


In [ ]:
body_types = auto_map([T for T in set([B.get('type') for S in itemdb for B in S.get('bodies',[]) ])])
prettyprint(body_types)

In [ ]:
terraform_types = auto_map([T for T in set([B.get('terraformingState') for S in itemdb for B in S.get('bodies',[]) if B.get('terraformingState') ]) ])
prettyprint(terraform_types)

In [ ]:
recreate_map(terraform_types)

In [ ]:
terraform_types

In [ ]:
planet_class = auto_map(set([B.get('subType') for S in itemdb for B in S.get('bodies',[]) if B.get('type') == 'Planet']))
prettyprint(planet_class)

In [ ]:
planet_values = {
    'Water world': 415613,
    'Earth-like world': 1126206,
    'Ammonia world': 597762,
}
classes_valuable = set([planet_class.get(C) for C in planet_values])
prettyprint(classes_valuable)

In [ ]:
[planet_values.get(C,0) for C in planet_class]

In [ ]:
planet_class_valuable = auto_map(set([
    B.get('subType') 
    for S in itemdb for B in S.get('bodies',[]) 
    if B.get('type') == 'Planet' and B.get('subType') in planet_values]))
prettyprint(planet_class_valuable)

In [ ]:
spectral_classes = auto_map([
    T for T in set(
        [B.get('spectralClass') for S in itemdb for B in S.get('bodies',[])  if B.get('spectralClass')] 
    )])
prettyprint(spectral_classes)

In [ ]:
luminosity_map = auto_map([
    T for T in set(
        [B.get('luminosity') for S in itemdb for B in S.get('bodies',[])  if B.get('luminosity')] 
    )])
prettyprint(luminosity_map)

In [ ]:
magnitude_map = auto_map([
    T for T in set(
        [int(round(B.get('absoluteMagnitude'))) for S in itemdb for B in S.get('bodies',[])  if B.get('absoluteMagnitude')] 
    )])
prettyprint(magnitude_map)

In [ ]:
        set([B.get('subType')[0] for S in itemdb for B in S.get('bodies',[])  if B.get('type') == 'Star' and B.get('subType')] )


In [ ]:
len(itemdb)

In [ ]:
set([
    B.get('subType') 
    for S in itemdb for B in S.get('bodies',[]) 
    if B.get('mainStar') and B.get('spectralClass') and B.get('spectralClass')[:-1] == 'CJ'])

In [ ]:
set([
    signal 
    for S in itemdb 
    for B in S.get('bodies',[]) 
    for signal in B.get('signals', {}).get('signals',{})  
    if B.get('type') == 'Planet'])

In [ ]:
set([B.get('luminosity') for S in itemdb for B in S.get('bodies',[]) if B.get('luminosity') ] )

In [ ]:
[x for x in itemdb][0]

In [ ]:
i=0

In [ ]:
i+=1

In [ ]:

[
    (
        
        round(B.get('solarMasses'),2),
        spectral_class_prefix.get(B.get('spectralClass')[:-1]),
        spectral_class_suffix.get(B.get('spectralClass')[-1])

    )  
    for B in itemdb[i].get('bodies',[])
    if B.get('mainStar')
]


In [ ]:

[
    (
        B.get('subType')[0],
        B.get('spectralClass'),
        B.get('solarMasses')

    )  
    for B in itemdb[i].get('bodies',[])
    if B.get('type') == 'Star'# and not B.get('mainStar')
]


In [ ]:

round(sum([
    B.get('solarMasses')

    for B in itemdb[i].get('bodies',[])
    if B.get('type') == 'Star'
]))


In [ ]:
#i+=1
sum([
    planet_values.get(B.get('subType'),0)

    for B in itemdb[i].get('bodies',[])
    if B.get('type') == 'Planet' 
])


In [ ]:
[
    (
        I.get('name'),
        I.get()
    for I in itemdb
    if sum([
        planet_values.get(B.get('subType'),0)

        for B in I.get('bodies',[])
        #if B.get('type') == 'Planet' 
    ]) > 0
]

In [ ]:

[
    B.get('subType')

    for B in itemdb[i].get('bodies',[])
    if B.get('type') == 'Planet' 
]


In [ ]:
prettyprint(itemdb[i])

